In [ ]:
import numpy as np
import pandas as pd
import subprocess
import random
import re
import h5py
import copy
import sys
from numba import njit
import matplotlib.pyplot as plt
from typing import List, Dict
import os
import glob
plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (12,10)
import json
import time
import matplotlib
from pathlib import Path
from scipy.interpolate import CubicSpline
spec_home="/home/himanshu/spec/my_spec"
matplotlib.matplotlib_fname()

# Various functions to read across levs
### Also functions to make reports

### Functions to read h5 files

In [ ]:
def read_profiler(file_name):
  with h5py.File(file_name,'r') as f:
    steps = set()
    procs = set()
    names = []
    f.visit(names.append)
    for name in names:
      step = name.split('.')[0][4:]
      steps.add(step)
      if 'Proc' in name:
        procs.add(name.split('/')[-1][4:-4])

    dict_list = []
    for step in steps:
      for proc in procs:
        data = f[f'Step{step}.dir/Proc{proc}.txt'][0].decode()

        lines = data.split("\n")
        time = float((lines[0].split("=")[-1])[:-1])

        curr_dict = {
            "t(M)": time,
            "step": step,
            "proc": proc
        }
        # Find where the columns end
        a = lines[4]
        event_end = a.find("Event")+5
        cum_end = a.find("cum(%)")+6
        exc_end = a.find("exc(%)")+6
        inc_end = a.find("inc(%)")+6

        for line in lines[6:-2]:
          Event = line[:event_end].strip()
          cum = float(line[event_end:cum_end].strip())
          exc = float(line[cum_end:exc_end].strip())
          inc = float(line[exc_end:inc_end].strip())
          N = int(line[inc_end:].strip())
          # print(a)
          # a = line.split("  ")
          # Event,cum,exc,inc,N = [i.strip() for i in a if i!= '']
          curr_dict[f'{Event}_cum'] = cum
          curr_dict[f'{Event}_exc'] = exc
          curr_dict[f'{Event}_inc'] = inc
          curr_dict[f'{Event}_N'] = N

        dict_list.append(curr_dict)
  return pd.DataFrame(dict_list)

def read_profiler_multiindex(folder_path:Path):
  dir_paths,dat_paths = list_all_dir_and_dat_files(folder_path)
  steps = set()
  # Get step names
  for dir in dir_paths:
    step = dir.name.split('.')[0][4:]
    steps.add(step)

  procs = set()
  # Get the proc names
  for txt in dir_paths[0].iterdir():
    if ".txt" in txt.name and "Summary" not in txt.name:
      procs.add(txt.name[4:-4])

  dict_list = []
  col_names = set()
  row_names = []
  for step in steps:
    for proc in procs:
      txt_file_path = folder_path/f'Step{step}.dir/Proc{proc}.txt'

      with txt_file_path.open("r") as f:
        lines = f.readlines()

      time = float((lines[0].split("=")[-1])[:-2])

      curr_dict = {
          "time": time,
          "step": step,
          "proc": proc
      }

      # Find where the columns end
      a = lines[4]
      event_end = a.find("Event")+5
      cum_end = a.find("cum(%)")+6
      exc_end = a.find("exc(%)")+6
      inc_end = a.find("inc(%)")+6

      row_names.append((str(proc),str(time)))

      for line in lines[6:-2]:
        Event = line[:event_end].strip()
        cum = float(line[event_end:cum_end].strip())
        exc = float(line[cum_end:exc_end].strip())
        inc = float(line[exc_end:inc_end].strip())
        N = int(line[inc_end:].strip())
        # print(a)
        # a = line.split("  ")
        # Event,cum,exc,inc,N = [i.strip() for i in a if i!= '']
        col_names.add(Event)
        curr_dict[("cum",Event)] = cum
        curr_dict[("exc",Event)] = exc
        curr_dict[("inc",Event)] = inc
        curr_dict[("N",Event)] = N

      dict_list.append(curr_dict)

  # Multi index rows
  index = pd.MultiIndex.from_tuples(row_names, names=["proc","t(M)"])
  df = pd.DataFrame(dict_list,index=index)
  
  # Multi index cols
  multi_index_columns = [(k if isinstance(k, tuple) else (k, '')) for k in df.columns]
  df.columns = pd.MultiIndex.from_tuples(multi_index_columns)
  df.columns.names = ['metric', 'process']

  # data.xs('24', level="proc")['N']
  # data.xs('0.511442', level="t(M)")['cum']
  # data.xs(('0','0.511442'),level=('proc','t(M)'))
  # data.xs('cum',level='metric',axis=1) = data['cum']
  # data.xs('MPI::MPreduceAdd(MV<double>)',level='process',axis=1)
  # data[data['time']<50]
  # data[data['time']<50]['cum'].xs('0',level='proc')['MPI::MPreduceAdd(MV<double>)']
  return df.sort_index()

### Functions to read dat and hist files

In [ ]:
def read_dat_file(file_name):
  cols_names = []
  # Read column names
  with open(file_name,'r') as f:
      lines = f.readlines()
      for line in lines:
        if "#" not in line:
          # From now onwards it will be all data
          break
        elif "=" in line:
          cols_names.append(line.split('=')[-1][1:-1].strip())
        else:
          continue

  return pd.read_csv(file_name,sep="\s+",comment="#",names=cols_names)

def hist_files_to_dataframe(file_path):
  # Function to parse a single line and return a dictionary of values
  def parse_line(line):
      data = {}
      # Find all variable=value pairs
      pairs = re.findall(r'([^;=\s]+)=\s*([^;]+)', line)
      for var, val in pairs:
          # Hist-GrDomain.txt should be parsed a little differently
          if 'ResizeTheseSubdomains' in var:
              items = val.split('),')
              items[-1] = items[-1][:-1]
              for item in items:
                name,_,vals = item.split("(")
                r,l,m=vals[:-1].split(',')
                data[f"{name}_R"] = int(r)
                data[f"{name}_L"] = int(l)
                data[f"{name}_M"] = int(m)
          else:
              data[var] = float(val) if re.match(r'^[\d.e+-]+$', val) else val
      return data
  
  with open(file_path, 'r') as file:
    # Parse the lines
    data = []
    for line in file.readlines():
        data.append(parse_line(line.strip()))

    # Create a DataFrame
    df = pd.DataFrame(data)

  return df

# Files like AhACoefs.dat have unequal number of columns
def read_dat_file_uneq_cols(file_name):
  cols_names = []

  temp_file = "./temp.csv"
  col_length = 0
  with open(file_name,'r') as f:
    with open(temp_file,'w') as w:
      lines = f.readlines()
      for line in lines:
        if(line[0] != '#'): # This is data
          w.writelines(" ".join(line.split()[:col_length])+"\n")
        if(line[0:3] == '# [' or line[0:4] == '#  ['): # Some dat files have comments on the top
          cols_names.append(line.split('=')[-1][1:-1].strip())
          col_length = col_length+1


  return pd.read_csv(temp_file,delim_whitespace=True,names=cols_names)

def read_dat_file_across_AA(file_pattern):

  path_pattern = file_pattern
  path_collection = []


  for folder_name in glob.iglob(path_pattern, recursive=True):
      if os.path.isdir(folder_name) or os.path.isfile(folder_name):
          path_collection.append(folder_name)
          print(folder_name)


  read_data_collection = []
  for path in path_collection:
    # AhACoefs.dat has uneq cols
    if "Coefs.dat" in path:
        read_data_collection.append(read_dat_file_uneq_cols(path))
    elif "Hist-" in path:
        read_data_collection.append(hist_files_to_dataframe(path))
    elif "Profiler" in path:
        read_data_collection.append(read_profiler(path))
    else:
        read_data_collection.append(read_dat_file(path))

  data = pd.concat(read_data_collection)
  rename_dict = {
     't':'t(M)',
     'time':'t(M)',
     'Time':'t(M)',
     'time after step':'t(M)',
  }
  data.rename(columns=rename_dict, inplace=True)
  # print(data.columns)
  return data

def read_AH_files(Ev_path):
  fileA = Ev_path + "Run/ApparentHorizons/AhA.dat"
  fileB = Ev_path + "Run/ApparentHorizons/AhB.dat"

  dataA = read_dat_file_across_AA(fileA)
  dataB = read_dat_file_across_AA(fileB)

  return dataA,dataB  

  
# Combines all the pvd files into a single file and save it in the base folder
def combine_pvd_files(base_folder:Path, file_pattern:str, output_path=None):
  pvd_start ="""<?xml version="1.0"?>\n<VTKFile type="Collection" version="0.1" byte_order="LittleEndian">\n  <Collection>\n"""
  pvd_end ="  </Collection>\n</VTKFile>"

  vis_folder_name = file_pattern.split("/")[-1][:-4]
  Lev = file_pattern[0:4]

  if output_path is None:
    output_path = f"{base_folder}/{vis_folder_name}_{Lev}.pvd"

  pvd_files = list(base_folder.glob(file_pattern))
  pvd_folders = list(base_folder.glob(file_pattern[:-4]))


  with open(output_path,'w') as write_file:
    write_file.writelines(pvd_start)
    for files in pvd_files:
      print(files)
      with files.open("r") as f:
        for line in f.readlines():
          line = line.replace(vis_folder_name,str(files)[:-4])
          if "DataSet" in line:
            write_file.writelines(line)
    write_file.writelines(pvd_end)
  
  print(output_path)

def moving_average(array,avg_len):
    return np.convolve(array,np.ones(avg_len))/avg_len
    
def moving_average_valid(array,avg_len):
    return np.convolve(array,np.ones(avg_len),'valid')/avg_len


def path_to_folder_name(folder_name):
  return folder_name.replace("/","_")

# Give a dict of {"run_name" = runs_path} and data_file_path to get {"run_name" = dat_file_data}
def load_data_from_levs(runs_path, data_file_path):
  data_dict = {}
  column_list = ""
  for run_name in runs_path.keys():
    data_dict[run_name] = read_dat_file_across_AA(runs_path[run_name]+data_file_path)
    column_list = data_dict[run_name].columns
  return column_list, data_dict

def add_diff_columns(runs_data_dict, x_axis, y_axis, diff_base):
  if diff_base not in runs_data_dict.keys():
    raise Exception(f"{diff_base} not in {runs_data_dict.keys()}")
  
  interpolated_data = CubicSpline(runs_data_dict[diff_base][x_axis],runs_data_dict[diff_base][y_axis],extrapolate=False)

  for key in runs_data_dict:
    if key == diff_base:
      continue
    df = runs_data_dict[key]
    df['diff_'+y_axis] = df[y_axis] - interpolated_data(df[x_axis])

def plot_graph_for_runs(runs_data_dict, x_axis, y_axis, minT, maxT, legend_dict=None, save_path=None, moving_avg_len=0, plot_fun = lambda x,y,label : plt.plot(x,y,label=label),sort_by=None, diff_base=None):
  sort_run_data_dict(runs_data_dict,sort_by=sort_by)
  current_runs_data_dict_keys = list(runs_data_dict.keys())

  if diff_base is not None:
    add_diff_columns(runs_data_dict, x_axis, y_axis, diff_base)
    current_runs_data_dict_keys = []
    for key in runs_data_dict:
      if key == diff_base:
        continue
      else:
        current_runs_data_dict_keys.append(key)
    y_axis = "diff_"+y_axis
 
  # Find the indices corresponding to maxT and minT
  minT_indx_list={}
  maxT_indx_list={}

  if legend_dict is None:
    legend_dict = {}
    for run_name in current_runs_data_dict_keys:
      legend_dict[run_name] = None
  else:
    for run_name in current_runs_data_dict_keys:
      if run_name not in legend_dict:
        raise ValueError(f"{run_name} not in {legend_dict=}")

  
  for run_name in current_runs_data_dict_keys:
    minT_indx_list[run_name] = len(runs_data_dict[run_name][x_axis][runs_data_dict[run_name][x_axis] < minT])
    maxT_indx_list[run_name] = len(runs_data_dict[run_name][x_axis][runs_data_dict[run_name][x_axis] < maxT])

  if moving_avg_len == 0:

    for run_name in current_runs_data_dict_keys:
      x_data = runs_data_dict[run_name][x_axis][minT_indx_list[run_name]:maxT_indx_list[run_name]]
      y_data = runs_data_dict[run_name][y_axis][minT_indx_list[run_name]:maxT_indx_list[run_name]]
      

    #   print(f"{len(x_data)=},{len(y_data)=},{len(np.argsort(x_data))=},{type(x_data)=}")

    #   sorted_indices = x_data.argsort()
    #   x_data = x_data.iloc[sorted_indices]
    #   y_data = y_data.iloc[sorted_indices]
      legend = legend_dict[run_name]
      if legend is None:
        legend = run_name
      plot_fun(x_data, y_data,legend)

    plt.xlabel(x_axis)
    plt.ylabel(y_axis)
    title = "\"" +  y_axis+"\" vs \""+x_axis+"\""
    if diff_base is not None:
      title = title + f" diff_base={diff_base}"
    plt.title(title)
    plt.legend()

  else:
    for run_name in current_runs_data_dict_keys:
      x_data = np.array(runs_data_dict[run_name][x_axis][minT_indx_list[run_name] + moving_avg_len-1:maxT_indx_list[run_name]])
      y_data = np.array(moving_average_valid(runs_data_dict[run_name][y_axis][minT_indx_list[run_name]:maxT_indx_list[run_name]], moving_avg_len))

    #   sorted_indices = np.argsort(x_data)
    #   x_data = x_data[sorted_indices]
    #   y_data = y_data[sorted_indices]
      legend = legend_dict[run_name]
      if legend is None:
        legend = run_name
      plot_fun(x_data, y_data,legend)

    plt.xlabel(x_axis)
    plt.ylabel(y_axis)
    title = "\"" + y_axis+ "\" vs \"" + x_axis + "\"  " + f"avg_window_len={moving_avg_len}"
    if diff_base is not None:
      title = title + f" diff_base={diff_base}"
    plt.title(title)
    plt.legend()

  
  if save_path is not None:
    fig_x_label = x_axis.replace("/","_").replace(".","_")
    fig_y_label = y_axis.replace("/","_").replace(".","_")
    save_file_name = f"{fig_y_label}_vs_{fig_x_label}_minT={minT}_maxT={maxT}".replace(".","_")
    if moving_avg_len > 0:
      save_file_name = save_file_name + f"_moving_avg_len={moving_avg_len}"
    if diff_base is not None:
      save_file_name = save_file_name + f"_diff_base={diff_base}"

    for run_name in current_runs_data_dict_keys:
      save_file_name = save_file_name + "__" + run_name.replace("/","_").replace(".","_")

    if len(save_file_name) >= 251: # <save_file_name>.png >=255
      save_file_name = save_file_name[:245]+str(random.randint(10000,99999))
      print(f"The filename was too long!! New filename is {save_file_name}")

    plt.savefig(save_path+save_file_name)


def find_file(pattern):
  return glob.glob(pattern, recursive=True)[0]

def plots_for_a_folder(things_to_plot,plot_folder_path,data_folder_path):
  for plot_info in things_to_plot:
    file_name = plot_info['file_name']
    y_arr = plot_info['columns'][1:]
    x_arr = [plot_info['columns'][0]]*len(y_arr)

    data = read_dat_file_across_AA(data_folder_path+"/**/"+file_name)
    plot_and_save(data,x_arr,y_arr,plot_folder_path,file_name)

def is_the_current_run_going_on(run_folder):
  if len(find_file(run_folder+"/**/"+"TerminationReason.txt")) > 0:
    return False
  else:
    return True

def plot_min_grid_spacing(runs_data_dict):
    '''
    runs_data_dict should have dataframes with MinimumGridSpacing.dat data.
    The function will compute the min grid spacing along all domains and plot it.
    '''
    keys = runs_data_dict.keys()
    if len(keys) == 0:
        print("There are no dataframes in the dict")

    for key in keys:
        t_step = runs_data_dict[key]['t']
        min_val = runs_data_dict[key].drop(columns=['t']).min(axis='columns')
        plt.plot(t_step,min_val,label=key)

    plt.legend()
    plt.xlabel("t")
    plt.ylabel("Min Grid Spacing")
    plt.title("Min grid spacing in all domains")
    plt.show()

def plot_GrAdjustSubChunksToDampingTimes(runs_data_dict):
    keys = runs_data_dict.keys()
    if len(keys) > 1:
        print("To plot the Tdamp for various quantities only put one dataframe in the runs_data_dict")

    data:pd.DataFrame = runs_data_dict[list(keys)[0]]
    tdamp_keys = []
    for key in data.keys():
        if 'Tdamp' in key:
            tdamp_keys.append(key)

    # Get a colormap
    cmap = plt.get_cmap('tab10')
    colors = cmap(np.linspace(0, 1, len(tdamp_keys)))

    t_vals = data['time']
    for i, color, key in zip(range(len(tdamp_keys)),colors, tdamp_keys):
        if i%2==0:
            plt.plot(t_vals,data[key],label=key,color=color)
        else:
            plt.plot(t_vals,data[key],label=key,color=color,linestyle="--")


    min_tdamp = data[tdamp_keys].min(axis='columns')
    plt.plot(t_vals,min_tdamp,label="min_tdamp",linewidth=3,linestyle="dotted",color="red")

    plt.legend()
    plt.xlabel("time")
    plt.title(list(keys)[0])
    plt.show()

def add_max_and_min_val(runs_data_dict):
    # If we load a file with 5 columns with first being time, then find max and min values for all the other columns, at all times and add it to the dataframe.
    # Useful when you want to find like Linf across all domains at all times
    for run_name in runs_data_dict.keys():
        data_frame = runs_data_dict[run_name]
        t = data_frame.iloc[:,0]
        max_val = np.zeros_like(t)
        min_val = np.zeros_like(t)
        for i in range(len(t)):
            max_val[i] = data_frame.iloc[i,1:].max()
            min_val[i] = data_frame.iloc[i,1:].max()

        # Add the values to the dataframe
        data_frame['max_val'] = max_val
        data_frame['min_val'] = min_val

def sort_run_data_dict(runs_data_dict:dict,sort_by=None):
    for run_name in runs_data_dict.keys():
        run_df = runs_data_dict[run_name]
        if sort_by is None:
            sort_by = run_df.keys()[0]
        runs_data_dict[run_name] = run_df.sort_values(by=sort_by)

## Functions to read horizon files

In [ ]:
def make_Bh_pandas(h5_dir):
    # Empty dataframe
    df = pd.DataFrame()
    
    # List of all the vars in the h5 file
    var_list = []
    h5_dir.visit(var_list.append)
    
    
    for var in var_list:
        # This means there is no time column
        # print(f"{var} : {h5_dir[var].shape}")
        if df.shape == (0,0):
            # data[:,0] is time and then we have the data
            data = h5_dir[var]
            
            # vars[:-4] to remove the .dat at the end
            col_names = make_col_names(var[:-4],data.shape[1]-1)
            col_names.append('t')
            # Reverse the list so that we get ["t","var_name"]
            col_names.reverse()            
            append_to_df(data[:],col_names,df)
            
        else:
            data = h5_dir[var]
            col_names = make_col_names(var[:-4],data.shape[1]-1)         
            append_to_df(data[:,1:],col_names,df)
            
    return df

def append_to_df(data,col_names,df):
    for i,col_name in enumerate(col_names):
        df[col_name] = data[:,i]
        
def make_col_names(val_name:str,val_size:int):
    col_names = []
    if val_size == 1:
        col_names.append(val_name)
    else:
        for i in range(val_size):
            col_names.append(val_name+f"_{i}")
    return col_names


def horizon_to_pandas(horizon_path:Path):
    assert(horizon_path.exists())
    df_dict = {}
    with h5py.File(horizon_path,'r') as hf:
        # Not all horizon files may have AhC
        for key in hf.keys():
            if key == 'VersionHist.ver':
                # Newer runs have this
                continue
            df_dict[key[:-4]] = make_Bh_pandas(hf[key])

    return df_dict

def read_horizon_across_Levs(path_list:List[Path]):
    df_listAB = []
    df_listC = []
    final_dict = {}
    for path in path_list:
        df_lev = horizon_to_pandas(path)
        # Either [AhA,AhB] or [AhA,AhB,AhC]
        if len(df_lev.keys()) > 1:
            df_listAB.append(df_lev)
        # Either [AhC] or [AhA,AhB,AhC]
        if (len(df_lev.keys()) == 1) or (len(df_lev.keys()) ==3):
            df_listC.append(df_lev)
    if len(df_listAB)==1:
        # There was only one lev
        final_dict = df_listAB[0]
    else:
        final_dict["AhA"] = pd.concat([df["AhA"] for df in df_listAB])
        final_dict["AhB"] = pd.concat([df["AhB"] for df in df_listAB])
        if len(df_listC) > 0:
            final_dict["AhC"] = pd.concat([df["AhC"] for df in df_listC])       
    
    return final_dict

def load_horizon_data_from_levs(base_path:Path, runs_path:Dict[str,Path]):
  data_dict = {}
  for run_name in runs_path.keys():
    path_list = list(base_path.glob(runs_path[run_name]))
    print(path_list)
    data_dict[run_name] = read_horizon_across_Levs(path_list)
  return data_dict

def flatten_dict(horizon_data_dict:Dict[str,pd.DataFrame]) -> Dict[str,pd.DataFrame] :
  flattened_data = {}
  for run_name in horizon_data_dict.keys():
      for horizons in horizon_data_dict[run_name]:
          flattened_data[run_name+"_"+horizons] = horizon_data_dict[run_name][horizons]
          # print(run_name+"_"+horizons)
  return flattened_data

# Plot dat files

In [ ]:
runs_to_plot = {}
# runs_to_plot["boost_ID_test_wrong"] =  "/groups/sxs/hchaudha/spec_runs/boost_ID_test_wrong/Ev/Lev3_A?/Run/"
# runs_to_plot["boost_ID_test_correct"] =  "/groups/sxs/hchaudha/spec_runs/boost_ID_test_correct/Ev/Lev3_A?/Run/"
# runs_to_plot["corrected_coord_spin1"] =  "/groups/sxs/hchaudha/spec_runs/corrected_coord_spin1/Ev/Lev3_A?/Run/"
# runs_to_plot["corrected_coord_spin2"] =  "/groups/sxs/hchaudha/spec_runs/corrected_coord_spin2/Ev/Lev3_A?/Run/"
# runs_to_plot["2_SpKS_q1_sA_0_0_0_sB_0_0_0_d15"] =  "/groups/sxs/hchaudha/spec_runs/2_SpKS_q1_sA_0_0_0_sB_0_0_0_d15/Ev/Lev3_A?/Run/"
# runs_to_plot["2_SpKS_q1_sA_0_0_0_sB_0_0_99_d15"] =  "/groups/sxs/hchaudha/spec_runs/2_SpKS_q1_sA_0_0_0_sB_0_0_99_d15/Ev/Lev3_A?/Run/"
# runs_to_plot["2_SpKS_q1_sA_0_0_0_sB_0_0_9_d15"] =  "/groups/sxs/hchaudha/spec_runs/2_SpKS_q1_sA_0_0_0_sB_0_0_9_d15/Ev/Lev3_A?/Run/"
# runs_to_plot["2_SpKS_q1_sA_0_0_9_sB_0_0_9_d15"] =  "/groups/sxs/hchaudha/spec_runs/2_SpKS_q1_sA_0_0_9_sB_0_0_9_d15/Ev/Lev3_A?/Run/"
runs_to_plot["3_DH_q1_ns_d18_L3"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3/Ev/Lev3_A?/Run/"
# runs_to_plot["L3_tol8_eq"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/tol8_eq/Lev3_A?/Run/"
# runs_to_plot["L3_tol9"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/tol9/Lev3_A?/Run/"
runs_to_plot["L3_tol10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/tol10/Lev3_A?/Run/"
# runs_to_plot["L3_tol10_hi"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/tol10_hi/Lev3_A?/Run/"
# runs_to_plot["L3_tol11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/tol11/Lev3_A?/Run/"
runs_to_plot["L3_all_100_tol10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/L6_tol10/Lev3_A?/Run/"
runs_to_plot["L3_all_1000_tol11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/L6_all_10_tol11/Lev3_A?/Run/"
# runs_to_plot["local_100_tol5_11"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/local_100_tol5_11/Lev3_A?/Run/"
# runs_to_plot["local_10_tol5_10"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/local_10_tol5_10/Lev3_A?/Run/"
# runs_to_plot["L3_local_10_tol5_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/local_10_tol5_10/Lev3_A?/Run/"
# runs_to_plot["L3_local_100_tol5_11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/local_100_tol5_11/Lev3_A?/Run/"

# runs_to_plot["high_accuracy_L3"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev3_A?/Run/"
# runs_to_plot["high_accuracy_L4"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev4_A?/Run/"
# runs_to_plot["high_accuracy_L5"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev5_A?/Run/"

# runs_to_plot["3_DH_q1_ns_d18_L3_rd"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3/Ev/Lev3_Ringdown/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L3_tol8_eq_rd"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/tol8_eq/Lev3_Ringdown/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L3_tol9_rd"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/tol9/Lev3_Ringdown/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L3_tol10_rd"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/tol10/Lev3_Ringdown/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L3_tol10_hi_rd"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/tol10_hi/Lev3_Ringdown/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L3_tol11_rd"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/tol11/Lev3_Ringdown/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L3_all_100_tol10_rd"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/L6_tol10/Lev3_Ringdown/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L3_all_1000_tol11_rd"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/L6_all_10_tol11/Lev3_Ringdown/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L3_rd"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3/Ev/Lev3_Ringdown/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L6"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L6/Ev/Lev6_A?/Run/"
# runs_to_plot["L6_1.1"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L6_higher_acc/L6_1.1/Lev6_A?/Run/"
# runs_to_plot["L6_1.1_dp8_tol_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L6_higher_acc/L6_1.1_dp8_tol_10/Lev6_A?/Run/"
# runs_to_plot["L6_1.1_tol_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L6_higher_acc/L6_1.1_tol_10/Lev6_A?/Run/"
# runs_to_plot["L6_tol_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L6_higher_acc/L6_tol_10/Lev6_A?/Run/"
# runs_to_plot["all_10"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_10/Lev3_A?/Run/"
# runs_to_plot["all_100"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100/Lev3_A?/Run/"
# runs_to_plot["near_bhs_10"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/near_bhs_10/Lev3_A?/Run/"
# runs_to_plot["near_bhs_100"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/near_bhs_100/Lev3_A?/Run/"
# runs_to_plot["same_obs"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/same_obs/Lev3_A?/Run/"
# runs_to_plot["all_10_obs"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_10_obs/Lev3_A?/Run/"
# runs_to_plot["all_100_obs"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_obs/Lev3_A?/Run/"
# runs_to_plot["all_10_obs_tol_10"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_10_obs_tol_10/Lev3_A?/Run/"
# runs_to_plot["all_100_obs_tol_10"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_obs_tol_10/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_1.1_b0"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_1.1_b0/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_1.1_b0_tol_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_1.1_b0_tol_10/Lev3_A?/Run/"
# runs_to_plot["all_100_1.1"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_1.1/Lev3_A?/Run/"
# runs_to_plot["all_100_1.1_dp8_tol_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_1.1_dp8_tol_10/Lev3_A?/Run/"
# runs_to_plot["all_100_1.1_tol_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_1.1_tol_10/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_2"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_2/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_2"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_2/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_3"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_3/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dp8"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dp8/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dp8_tol10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dp8_tol10/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dp8_tol11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dp8_tol11/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dt"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dt_0.02"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt_0.02/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dt_0.03"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt_0.03/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dt_0.04"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt_0.04/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dt_0.025"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt_0.025/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dt_0.021"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt_0.021/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dt_0.022"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt_0.022/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dt_0.023"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt_0.023/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dt_0.024"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt_0.024/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dt_0.0225"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt_0.0225/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dt005"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt0.005/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_tol_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_tol_10/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_tol_11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_tol_11/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_tol_9"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_tol_9/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_tol_1.128e-11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_tol_1.128e-11/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_tol_1.692e-11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_tol_1.692e-11/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_tol_3.383e-11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_tol_3.383e-11/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690/Lev3_A?/Run/"
# runs_to_plot["near_bhs_10_obs"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/near_bhs_10_obs/Lev3_A?/Run/"
# runs_to_plot["near_bhs_100_obs"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/near_bhs_100_obs/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L6_AA"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L6_AA/Ev/Lev3_A?/Run/"
# runs_to_plot["4_SphKS_q1_15_SSphKS_ID"] =  "/groups/sxs/hchaudha/spec_runs/4_SphKS_q1_15_SSphKS_ID/Ev/Lev3_A?/Run/"
# runs_to_plot["4_SphKS_q1_15_SKS_ID"] =  "/groups/sxs/hchaudha/spec_runs/4_SphKS_q1_15_SKS_ID/Ev/Lev3_A?/Run/"
# runs_to_plot["5_gd_SphKS_gauge_ID"] =  "/groups/sxs/hchaudha/spec_runs/5_gd_SphKS_gauge_ID/Ev/Lev2_A[A-S]/Run/"
# runs_to_plot["5_ngd_SphKS_ID"] =  "/groups/sxs/hchaudha/spec_runs/5_ngd_SphKS_ID/Ev/Lev2_A?/Run/"
# runs_to_plot["5_ngd_KS_ID"] =  "/groups/sxs/hchaudha/spec_runs/5_ngd_KS_ID/Ev/Lev2_A?/Run/"

# runs_to_plot["all_100_t2690_eteq_tol_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eteq_tol_10/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eteq_tol_11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eteq_tol_11/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eteq_tol_12"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eteq_tol_12/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eteq_tol_8"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eteq_tol_8/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eteq_tol_9"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eteq_tol_9/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eteq_tol_eq"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eteq_tol_eq/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eth_tol_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eth_tol_10/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eth_tol_11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eth_tol_11/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eth_tol_12"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eth_tol_12/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eth_tol_8"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eth_tol_8/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eth_tol_9"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eth_tol_9/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eth_tol_eq"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eth_tol_eq/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_etl_tol_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_etl_tol_10/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_etl_tol_11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_etl_tol_11/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_etl_tol_8"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_etl_tol_8/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_etl_tol_9"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_etl_tol_9/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_etl_tol_eq"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_etl_tol_eq/Lev3_A?/Run/"

# runs_to_plot["t6115_tol11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol11/Lev3_A?/Run/"
# runs_to_plot["t6115_tol10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol10/Lev3_A?/Run/"
# runs_to_plot["t6115_tol9"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol9/Lev3_A?/Run/"
# runs_to_plot["t6115_tol8"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol8/Lev3_A?/Run/"
# runs_to_plot["t6115_tol7"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol7/Lev3_A?/Run/"
# runs_to_plot["t6115_tol11_AMR"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol11_AMR/Lev3_A?/Run/"
# runs_to_plot["t6115_tol10_AMR"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol10_AMR/Lev3_A?/Run/"
# runs_to_plot["t6115_tol9_AMR"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol9_AMR/Lev3_A?/Run/"
# runs_to_plot["t6115_tol8_AMR"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol8_AMR/Lev3_A?/Run/"
# runs_to_plot["t6115_tol7_AMR"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol7_AMR/Lev3_A?/Run/"
# runs_to_plot["t6115_tol8_linf"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol8_linf/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.02"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.02/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.03"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.03/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.041"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.041/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.042"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.042/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.043"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.043/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.044"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.044/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.045"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.045/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.046"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.046/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.047"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.047/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.048"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.048/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.049"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.049/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.050"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.050/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.052"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.052/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.054"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.054/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.056"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.056/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_2.368e-07"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_2.368e-07/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_1.692e-07"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_1.692e-07/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_1.015e-07"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_1.015e-07/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_6.767e-08"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_6.767e-08/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_5.075e-08"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_5.075e-08/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_3.383e-08"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_3.383e-08/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_2.256e-08"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_2.256e-08/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_1.692e-08"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_1.692e-08/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_1.128e-08"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_1.128e-08/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_6.767e-09"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_6.767e-09/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_4.833e-09"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_4.833e-09/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_3.383e-09"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_3.383e-09/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_1.692e-09"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_1.692e-09/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_1.128e-09"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_1.128e-09/Lev3_A?/Run/"


# runs_to_plot["all_100_t2710_0.021_0.021"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2710_0.021_0.21/Lev3_AE/Run/"
# runs_to_plot["all_100_t2710_0.021_0.022"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2710_0.021_0.22/Lev3_AE/Run/"
# runs_to_plot["all_100_t2710_0.021_0.0225"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2710_0.021_0.225/Lev3_AE/Run/"
# runs_to_plot["all_100_t2710_0.021_0.023"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2710_0.021_0.23/Lev3_AE/Run/"
# runs_to_plot["all_100_t2710_0.021_0.024"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2710_0.021_0.24/Lev3_AE/Run/"
# runs_to_plot["all_100_t2710_0.021_max_tol8"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2710_0.021_max_tol8/Lev3_AE/Run/"
# runs_to_plot["all_100_t2710_0.021_max_tol9"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2710_0.021_max_tol9/Lev3_AE/Run/"
# runs_to_plot["all_100_t2710_0.021_max_tol10"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2710_0.021_max_tol10/Lev3_AE/Run/"
# runs_to_plot["all_100_t2710_0.021_max_tol10.5"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2710_0.021_max_tol10.5/Lev3_AE/Run/"
# runs_to_plot["all_100_t2710_0.021_max_tol11"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2710_0.021_max_tol11/Lev3_AE/Run/"

# runs_to_plot["eq_t4000_tol10"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_all_10/eq_t4000_tol10/Lev3_A?/Run/"
# runs_to_plot["eq_t4000_tol5_10"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_all_10/eq_t4000_tol5_10/Lev3_A?/Run/"
# runs_to_plot["eq_t4000_tol5_11"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_all_10/eq_t4000_tol5_11/Lev3_A?/Run/"
# runs_to_plot["eq_t4000_tol9"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_all_10/eq_t4000_tol9/Lev3_A?/Run/"
# runs_to_plot["t4000_tol10"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_all_10/t4000_tol10/Lev3_A?/Run/"
# runs_to_plot["t4000_tol5_10"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_all_10/t4000_tol5_10/Lev3_A?/Run/"
# runs_to_plot["t4000_tol5_11"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_all_10/t4000_tol5_11/Lev3_A?/Run/"
# runs_to_plot["t4000_tol8"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_all_10/t4000_tol8/Lev3_A?/Run/"
# runs_to_plot["t4000_tol9"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_all_10/t4000_tol9/Lev3_A?/Run/"

# runs_to_plot["Lev3_AA_tol10_all_10"]  = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/rd_all_10_tol11/Lev3_AA_tol10_all_10/Lev3_A?_/Run/"
# runs_to_plot["Lev3_AA_tol11_all_10"]  = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/rd_all_10_tol11/Lev3_AA_tol11_all_10/Lev3_A?_/Run/"
# runs_to_plot["Lev3_AA_tol12_all_10"]  = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/rd_all_10_tol11/Lev3_AA_tol12_all_10/Lev3_A?_/Run/"
# runs_to_plot["Lev3_AA_tol5_10_all_10"]  = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/rd_all_10_tol11/Lev3_AA_tol5_10_all_10/Lev3_A?_/Run/"
# runs_to_plot["Lev3_AA_tol5_11_all_10"]  = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/rd_all_10_tol11/Lev3_AA_tol5_11_all_10/Lev3_A?_/Run/"
# runs_to_plot["Lev3_AA_tol10"]  = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/rd_all_10_tol11/Lev3_AA_tol10/Lev3_A?_/Run/"
# runs_to_plot["Lev3_AA_tol11"]  = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/rd_all_10_tol11/Lev3_AA_tol11/Lev3_A?_/Run/"
# runs_to_plot["Lev3_AA_tol12"]  = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/rd_all_10_tol11/Lev3_AA_tol12/Lev3_A?_/Run/"
# runs_to_plot["Lev3_AA_tol5_10"]  = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/rd_all_10_tol11/Lev3_AA_tol5_10/Lev3_A?_/Run/"
# runs_to_plot["Lev3_AA_tol5_11"]  = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/rd_all_10_tol11/Lev3_AA_tol5_11/Lev3_A?_/Run/"

runs_to_plot["3_100"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/changing_spectral_grid/3_100/Lev3_A?/Run/"
runs_to_plot["3_10"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/changing_spectral_grid/3_10/Lev3_A?/Run/"
runs_to_plot["3_1"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/changing_spectral_grid/3_1/Lev3_A?/Run/"
runs_to_plot["2_100"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/changing_spectral_grid/2_100/Lev3_A?/Run/"
runs_to_plot["2_10"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/changing_spectral_grid/2_10/Lev3_A?/Run/"
runs_to_plot["2_1"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/changing_spectral_grid/2_1/Lev3_A?/Run/"
runs_to_plot["1_100"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/changing_spectral_grid/1_100/Lev3_A?/Run/"
# runs_to_plot["1_10"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/changing_spectral_grid/1_10/Lev3_A?/Run/"

# data_file_path = "ConstraintNorms/GhCe.dat"
# data_file_path = "ConstraintNorms/GhCeExt.dat"
# data_file_path = "ConstraintNorms/GhCeExt_L2.dat"
# data_file_path = "ConstraintNorms/GhCeExt_Norms.dat"
# data_file_path = "ConstraintNorms/GhCe_L2.dat"
# data_file_path = "ConstraintNorms/GhCe_Linf.dat"
data_file_path = "ConstraintNorms/GhCe_Norms.dat"
# data_file_path = "ConstraintNorms/GhCe_VolL2.dat"
# data_file_path = "ConstraintNorms/NormalizedGhCe_Linf.dat"
# data_file_path = "ConstraintNorms/NormalizedGhCe_Norms.dat"
# data_file_path="CharSpeedNorms/CharSpeeds_Min_SliceLFF.SphereA0.dat"
# data_file_path="MinimumGridSpacing.dat"
# data_file_path="GrAdjustMaxTstepToDampingTimes.dat"
# data_file_path="GrAdjustSubChunksToDampingTimes.dat"
# data_file_path="DiagAhSpeedA.dat"
# data_file_path="ApparentHorizons/AhA.dat"
# data_file_path="ApparentHorizons/AhB.dat" 
# data_file_path="ApparentHorizons/MinCharSpeedAhA.dat"
# data_file_path="ApparentHorizons/RescaledRadAhA.dat"
# data_file_path="ApparentHorizons/AhACoefs.dat"
# data_file_path="ApparentHorizons/AhBCoefs.dat"
# data_file_path="ApparentHorizons/Trajectory_AhB.dat"
# data_file_path="ApparentHorizons/HorizonSepMeasures.dat"
# data_file_path="TStepperDiag.dat"
# data_file_path="TimeInfo.dat"
# data_file_path = "Hist-FuncSkewAngle.txt"
# data_file_path = "Hist-FuncCutX.txt"
# data_file_path = "Hist-FuncExpansionFactor.txt"
# data_file_path = "Hist-FuncLambdaFactorA0.txt"
# data_file_path = "Hist-FuncLambdaFactorA.txt"
# data_file_path = "Hist-FuncLambdaFactorB0.txt"
# data_file_path = "Hist-FuncLambdaFactorB.txt"
# data_file_path = "Hist-FuncQuatRotMatrix.txt"
# data_file_path = "Hist-FuncSkewAngle.txt"
# data_file_path = "Hist-FuncSmoothCoordSep.txt"
# data_file_path = "Hist-FuncSmoothMinDeltaRNoLam00AhA.txt"
# data_file_path = "Hist-FuncSmoothMinDeltaRNoLam00AhB.txt"
# data_file_path = "Hist-FuncSmoothRAhA.txt"
# data_file_path = "Hist-FuncSmoothRAhB.txt"
# data_file_path = "Hist-FuncTrans.txt"
# data_file_path = "Hist-GrDomain.txt"
# data_file_path = "Profiler.h5"
column_names, runs_data_dict = load_data_from_levs(runs_to_plot,data_file_path)
print(column_names)

In [ ]:
moving_avg_len=0
save_path = None
diff_base = None
x_axis = 't(M)'

# diff_base = 'Lev3_AA_tol12_all_10'
# add_max_and_min_val(runs_data_dict)
# y_axis = 'max_val'
# y_axis = 'min_val'

y_axis = 'Linf(GhCe)'
# y_axis = 'Tx'
# y_axis = 'Linf(GhCeExt)'
# y_axis = 'L2(GhCe)'
# y_axis = 'Linf(NormalizedGhCe) on SphereA0'
# y_axis = 'Linf(GhCe) on SphereA0'
# y_axis = 'Linf(GhCe) on SphereB0'
# y_axis = 'Linf(GhCe) on SphereC0'
# y_axis = 'Linf(GhCe) on SphereE0'
# y_axis = 'L2(GhCe) on SphereB0'
# y_axis = 'Linf(GhCe) on SphereC1'
# y_axis = 'Linf(GhCe) on CylinderSMB1.0'
# y_axis = 'Linf(GhCe) on CylinderSMB1.0'
# y_axis = 'Linf(GhCe) on FilledCylinderMB0'
# y_axis = 'Linf(GhCe) on FilledCylinderMA0'

# y_axis = 'MPI::MPreduceAdd(MV<double>)_exc'
# x_axis = 't'
# y_axis = 'T [hours]'
# y_axis = 'dt/dT'

# x_axis = 't'
# y_axis = 'CoordSepHorizons'
# y_axis = 'ProperSepHorizons'
# y_axis = 'MinCharSpeedAhA[9]'
# y_axis = 'MinimumGridSpacing[SphereB0]'
# y_axis = 'MinimumGridSpacing[SphereA0]'
# y_axis = 'MinimumGridSpacing[CylinderSMB0.0]'
# y_axis = 'dt/dT'


# x_axis = 'time'
# y_axis = 'ROverRAh(surf 9)'
# y_axis = 'L_surface'
# y_axis = 'L_mesh'
# y_axis = 'L_max'
# y_axis = 'NormalizedResidual'
# y_axis = 'Shape_TruncationError'
# y_axis = 'NumIterations'
# y_axis = 'max(R_ij)'
# y_axis = 'Coef(8,8)'
# y_axis = 'Coef(1,-1)'
# y_axis = 'Center-y'
# y_axis = 'Coef(0,0)'
# y_axis = 'Shape_NumberOfPiledUpModes'
# y_axis = 'max(r)'
# y_axis = 'min(r)'
# y_axis = 'max(|r^i-c^i|)'


# y_axis = 'courant factor'
# y_axis = 'error/1e-08'
# y_axis = 'NfailedSteps'
# y_axis = 'NumRhsEvaluations in this segment'
# y_axis = 'dt'

minT = 0
# minT = 1450
# minT = 2100
# minT = 2692
# minT = 2710
# minT = 3500
# minT = 4000
# minT = 6115+5
minT = 9272
# minT = 9100
# minT = 9372

# maxT = 2710
# maxT = minT+0.5
# maxT = minT+2
# maxT = minT+100
# maxT = 4500
# maxT = 4000
# maxT = 9300
# maxT = 9375
maxT = 40000
# moving_avg_len = 50
# moving_avg_len = 10

plot_fun = lambda x,y,label : plt.plot(x,y,label=label)
# plot_fun = lambda x,y,label : plt.plot(x,y,label=label,marker='x')
plot_fun = lambda x,y,label : plt.semilogy(x,y,label=label)
# plot_fun = lambda x,y,label : plt.semilogy(x,y,label=label,marker='x')
# plot_fun = lambda x,y,label : plt.loglog(x,y,label=label) 
# plot_fun = lambda x,y,label : plt.scatter(x,y,label=label,s=10,marker="x")
# save_path = "/groups/sxs/hchaudha/rough/high_acc_plots/"
# save_path = "/groups/sxs/hchaudha/rough/plots/"
# save_path = "/home/hchaudha/notes/spec_accuracy/figures/"
legend_dict = {}
for key in runs_data_dict.keys():
  legend_dict[key] = None

# legend_dict = { '3_DH_q1_ns_d18_L3': "Lev3",
#                 '3_DH_q1_ns_d18_L6': "Lev6",
#                 'all_10': "Lev3_all_tols_10",
#                 'all_100': "Lev3_all_tols_100",
#                 'near_bhs_10': "Lev3_sphere_AB_tols_10",
#                 'near_bhs_100': "Lev3_sphere_AB_tols_100"}
# legend_dict = {
#  '3_DH_q1_ns_d18_L3':"Lev3_ode_tol_1e-8",
#  '3_DH_q1_ns_d18_L3_tol9':"Lev3_ode_tol_1e-9",
#  '3_DH_q1_ns_d18_L3_tol10':"Lev3_ode_tol_1e-10",
#  '3_DH_q1_ns_d18_L3_tol11':"Lev3_ode_tol_1e-11",
#  '3_DH_q1_ns_d18_L3_all_100_tol10':"Lev3_AMR_tol_100_ode_tol_1e-11",
#  }

with plt.style.context('default'):
  plt.rcParams["figure.figsize"] = (12,10)
  plt.rcParams["figure.autolayout"] = True
  plot_graph_for_runs(runs_data_dict, x_axis, y_axis, minT, maxT, legend_dict=legend_dict, save_path=save_path, moving_avg_len=moving_avg_len, plot_fun=plot_fun, diff_base=diff_base)
  plt.show()

In [ ]:
moving_avg_len=0
save_path = "/home/hchaudha/notes/spec_accuracy/ghce_domains/"
diff_base = None

plot_fun = lambda x,y,label : plt.semilogy(x,y,label=label,marker='x')
plot_fun = lambda x,y,label : plt.semilogy(x,y,label=label)

legend_dict = {}
for key in runs_data_dict.keys():
  legend_dict[key] = None

minT = 9300
maxT = 10000

x_axis = 't(M)'

for y_axis in column_names:
  with plt.style.context('default'):
    plt.rcParams["figure.figsize"] = (12,10)
    plt.rcParams["figure.autolayout"] = True
    plot_graph_for_runs(runs_data_dict, x_axis, y_axis, minT, maxT, legend_dict=legend_dict, save_path=save_path, moving_avg_len=moving_avg_len, plot_fun=plot_fun, diff_base=diff_base)
    plt.close()
  print(f"{y_axis} done!")

In [ ]:
for i in column_names[1:]:
  print(i)

In [ ]:
df = runs_data_dict['all_100_t2690_obs']
minT = 2690
minT = 2700
maxT = 2710
maxT = minT+2
df = df[df['time']>minT]
df = df[df['time']<maxT]
from scipy.signal import find_peaks

# plt.plot(df['time'],df['Linf(GhCe) on SphereA0'],marker='x')
# plt.plot(df['time'],df['Linf(GhCe) on SphereA4'],marker='x')
plt.plot(df['time'],df['Linf(GhCe) on SphereA3'],marker='x')
# plt.yscale('log')
plt.show()

In [ ]:
data = runs_data_dict['all_100_t2690_obs_grid_tol_10']
data = data[data['time after step']>2690]
dt_arr = np.array(data.dt)
averaged_dt = np.zeros_like(dt_arr)
averaged_dt[0] =  dt_arr.mean() 
N = 100
for i in range(len(dt_arr)-1):
  averaged_dt[i+1] = averaged_dt[i]*(N-1)/(N)+dt_arr[i+1]*1/N

In [ ]:
plt.plot(data['time after step'],averaged_dt)
plt.plot(data['time after step'],dt_arr)
plt.show()

In [ ]:
# data_file_path="ConstraintNorms/GhCe_L2.dat"
data_file_path="ConstraintNorms/GhCe_Linf.dat"
# data_file_path="ConstraintNorms/NormalizedGhCe_Linf.dat"
# data_file_path="ConstraintNorms/GhCe_VolL2.dat"
column_names_linf2, runs_data_dict_linf2 = load_data_from_levs(runs_to_plot,data_file_path)
# print(column_names_linf2)
print(runs_data_dict_linf2.keys())

In [ ]:
# run_name = list(runs_data_dict_linf2.keys())[4]
run_name = 'high_accuracy_L5'
run_name = '3_DH_q1_ns_d18_L6'
# run_name = '3_DH_q1_ns_d18_L3_all_100_tol10_rd'
# run_name = 'all_100_t2690_eteq_tol_11'
# run_name = 'all_100_obs_tol_10'
# run_name = 'all_10_obs'
# run_name = 'all_10_obs_tol_10'
# run_name = 'all_100_t2690_obs_grid_dt'
df = runs_data_dict_linf2[run_name].copy()
df = df.sort_values(by=df.columns[0])

tmin= 1050
# tmin=9300
# tmin=9372
# tmin=2691
tmax=2000
# tmax=tmin+4
# tmax=6150
# tmax=50000

df = df[df['t(M)']>=tmin]
df = df[df['t(M)']<tmax]
t_name = df.columns[0]
y_axis = df.columns[1].split(" ")[0]
all_cols_but_t = df.columns[1:]
# Find the maximum value across columns B, C, D, and F for each row
df['max_val'] = df[all_cols_but_t].max(axis=1)

# Determine which column had the maximum value
df['max_source'] = df[all_cols_but_t].idxmax(axis=1)

# List all columns that have at least one max value
columns_with_max = df['max_source'].unique()

# Generate a colormap for the columns with at least one max value
num_colors = len(columns_with_max)
colors = plt.get_cmap('tab20', num_colors)  # Using 'tab20' colormap
color_map = {column: colors(i) for i, column in enumerate(columns_with_max)}

# Plot max_BCD vs t with different colors for different sources
plt.figure(figsize=(18, 10))
for i,source in enumerate(columns_with_max):
    subset = df[df['max_source'] == source]
    if i%4 == 0:
        plt.scatter(subset[t_name], subset['max_val'], color=color_map[source], label=source, s=10, marker="^")
    if i%4 == 1:
        plt.scatter(subset[t_name], subset['max_val'], color=color_map[source], label=source, s=10, marker="v")
    if i%4 == 2:
        plt.scatter(subset[t_name], subset['max_val'], color=color_map[source], label=source, s=10, marker=">")
    if i%4 == 3:
        plt.scatter(subset[t_name], subset['max_val'], color=color_map[source], label=source, s=10, marker="<")

plt.xlabel(t_name)
plt.ylabel(y_axis)
plt.yscale('log')
plt.title(f'{y_axis} vs {t_name} for {run_name}')
plt.legend()
plt.grid(True)  
plt.tight_layout()
plt.savefig(f"/groups/sxs/hchaudha/rough/{run_name}_{y_axis}_vs_{t_name}_{tmin}_{tmax}.png", dpi=500)
plt.show()

In [ ]:
data_file_path="MinimumGridSpacing.dat"
column_names, runs_data_dict = load_data_from_levs(runs_to_plot,data_file_path)
sort_run_data_dict(runs_data_dict)
plot_min_grid_spacing(runs_data_dict)

In [ ]:
data_file_path="GrAdjustSubChunksToDampingTimes.dat"
column_names, runs_data_dict = load_data_from_levs(runs_to_plot,data_file_path)
sort_run_data_dict(runs_data_dict)

In [ ]:
print(runs_data_dict.keys())
idx = 0
runs_data_dict_temp = {f"{list(runs_data_dict.keys())[idx]}":runs_data_dict[list(runs_data_dict.keys())[idx]]}
plot_GrAdjustSubChunksToDampingTimes(runs_data_dict_temp)

# plots for h5 files

In [ ]:
runs_to_plot = {}
# runs_to_plot["boost_ID_test_wrong"] =  "/groups/sxs/hchaudha/spec_runs/boost_ID_test_wrong/Ev/Lev3_A?/Run/"
# runs_to_plot["boost_ID_test_correct"] =  "/groups/sxs/hchaudha/spec_runs/boost_ID_test_correct/Ev/Lev3_A?/Run/"
# runs_to_plot["corrected_coord_spin1"] =  "/groups/sxs/hchaudha/spec_runs/corrected_coord_spin1/Ev/Lev3_A?/Run/"
# runs_to_plot["corrected_coord_spin2"] =  "/groups/sxs/hchaudha/spec_runs/corrected_coord_spin2/Ev/Lev3_A?/Run/"
# runs_to_plot["2_SpKS_q1_sA_0_0_0_sB_0_0_0_d15"] =  "/groups/sxs/hchaudha/spec_runs/2_SpKS_q1_sA_0_0_0_sB_0_0_0_d15/Ev/Lev3_A?/Run/"
# runs_to_plot["2_SpKS_q1_sA_0_0_0_sB_0_0_99_d15"] =  "/groups/sxs/hchaudha/spec_runs/2_SpKS_q1_sA_0_0_0_sB_0_0_99_d15/Ev/Lev3_A?/Run/"
# runs_to_plot["2_SpKS_q1_sA_0_0_0_sB_0_0_9_d15"] =  "/groups/sxs/hchaudha/spec_runs/2_SpKS_q1_sA_0_0_0_sB_0_0_9_d15/Ev/Lev3_A?/Run/"
# runs_to_plot["2_SpKS_q1_sA_0_0_9_sB_0_0_9_d15"] =  "/groups/sxs/hchaudha/spec_runs/2_SpKS_q1_sA_0_0_9_sB_0_0_9_d15/Ev/Lev3_A?/Run/"
runs_to_plot["3_DH_q1_ns_d18_L3"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3/Ev/Lev3_A?/Run/"
# runs_to_plot["L3_tol8_eq"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/tol8_eq/Lev3_A?/Run/"
# runs_to_plot["L3_tol9"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/tol9/Lev3_A?/Run/"
# runs_to_plot["L3_tol10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/tol10/Lev3_A?/Run/"
# runs_to_plot["L3_tol10_hi"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/tol10_hi/Lev3_A?/Run/"
# runs_to_plot["L3_tol11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/tol11/Lev3_A?/Run/"
# runs_to_plot["L3_all_100_tol10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/L6_tol10/Lev3_A?/Run/"
# runs_to_plot["L3_all_1000_tol11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/L6_all_10_tol11/Lev3_A?/Run/"
# runs_to_plot["local_100_tol5_11"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/local_100_tol5_11/Lev3_A?/Run/"
# runs_to_plot["local_10_tol5_10"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/local_10_tol5_10/Lev3_A?/Run/"
# runs_to_plot["L3_local_10_tol5_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/local_10_tol5_10/Lev3_A?/Run/"
# runs_to_plot["L3_local_100_tol5_11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/local_100_tol5_11/Lev3_A?/Run/"

# runs_to_plot["high_accuracy_L3"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev3_A?/Run/"
# runs_to_plot["high_accuracy_L4"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev4_A?/Run/"
# runs_to_plot["high_accuracy_L5"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev5_A?/Run/"

# runs_to_plot["3_DH_q1_ns_d18_L3_rd"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3/Ev/Lev3_Ringdown/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L3_tol8_eq_rd"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/tol8_eq/Lev3_Ringdown/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L3_tol9_rd"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/tol9/Lev3_Ringdown/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L3_tol10_rd"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/tol10/Lev3_Ringdown/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L3_tol10_hi_rd"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/tol10_hi/Lev3_Ringdown/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L3_tol11_rd"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/tol11/Lev3_Ringdown/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L3_all_100_tol10_rd"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/L6_tol10/Lev3_Ringdown/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L3_all_1000_tol11_rd"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/L6_all_10_tol11/Lev3_Ringdown/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L3_rd"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3/Ev/Lev3_Ringdown/Lev3_A?/Run/"
runs_to_plot["3_DH_q1_ns_d18_L6"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L6/Ev/Lev6_A?/Run/"
# runs_to_plot["L6_1.1"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L6_higher_acc/L6_1.1/Lev6_A?/Run/"
# runs_to_plot["L6_1.1_dp8_tol_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L6_higher_acc/L6_1.1_dp8_tol_10/Lev6_A?/Run/"
# runs_to_plot["L6_1.1_tol_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L6_higher_acc/L6_1.1_tol_10/Lev6_A?/Run/"
# runs_to_plot["L6_tol_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L6_higher_acc/L6_tol_10/Lev6_A?/Run/"
# runs_to_plot["all_10"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_10/Lev3_A?/Run/"
# runs_to_plot["all_100"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100/Lev3_A?/Run/"
# runs_to_plot["near_bhs_10"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/near_bhs_10/Lev3_A?/Run/"
# runs_to_plot["near_bhs_100"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/near_bhs_100/Lev3_A?/Run/"
# runs_to_plot["same_obs"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/same_obs/Lev3_A?/Run/"
runs_to_plot["all_10_obs"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_10_obs/Lev3_A?/Run/"
runs_to_plot["all_100_obs"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_obs/Lev3_A?/Run/"
runs_to_plot["all_10_obs_tol_10"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_10_obs_tol_10/Lev3_A?/Run/"
runs_to_plot["all_100_obs_tol_10"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_obs_tol_10/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_1.1_b0"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_1.1_b0/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_1.1_b0_tol_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_1.1_b0_tol_10/Lev3_A?/Run/"
# runs_to_plot["all_100_1.1"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_1.1/Lev3_A?/Run/"
# runs_to_plot["all_100_1.1_dp8_tol_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_1.1_dp8_tol_10/Lev3_A?/Run/"
# runs_to_plot["all_100_1.1_tol_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_1.1_tol_10/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_2"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_2/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_2"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_2/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_3"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_3/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dp8"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dp8/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dp8_tol10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dp8_tol10/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dp8_tol11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dp8_tol11/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dt"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dt_0.02"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt_0.02/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dt_0.03"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt_0.03/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dt_0.04"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt_0.04/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dt_0.025"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt_0.025/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dt_0.021"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt_0.021/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dt_0.022"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt_0.022/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dt_0.023"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt_0.023/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dt_0.024"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt_0.024/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dt_0.0225"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt_0.0225/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dt005"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt0.005/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_tol_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_tol_10/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_tol_11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_tol_11/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_tol_9"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_tol_9/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_tol_1.128e-11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_tol_1.128e-11/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_tol_1.692e-11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_tol_1.692e-11/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_tol_3.383e-11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_tol_3.383e-11/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690/Lev3_A?/Run/"
# runs_to_plot["near_bhs_10_obs"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/near_bhs_10_obs/Lev3_A?/Run/"
# runs_to_plot["near_bhs_100_obs"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/near_bhs_100_obs/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L6_AA"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L6_AA/Ev/Lev3_A?/Run/"
# runs_to_plot["4_SphKS_q1_15_SSphKS_ID"] =  "/groups/sxs/hchaudha/spec_runs/4_SphKS_q1_15_SSphKS_ID/Ev/Lev3_A?/Run/"
# runs_to_plot["4_SphKS_q1_15_SKS_ID"] =  "/groups/sxs/hchaudha/spec_runs/4_SphKS_q1_15_SKS_ID/Ev/Lev3_A?/Run/"
# runs_to_plot["5_gd_SphKS_gauge_ID"] =  "/groups/sxs/hchaudha/spec_runs/5_gd_SphKS_gauge_ID/Ev/Lev2_A[A-S]/Run/"
# runs_to_plot["5_ngd_SphKS_ID"] =  "/groups/sxs/hchaudha/spec_runs/5_ngd_SphKS_ID/Ev/Lev2_A?/Run/"
# runs_to_plot["5_ngd_KS_ID"] =  "/groups/sxs/hchaudha/spec_runs/5_ngd_KS_ID/Ev/Lev2_A?/Run/"

# runs_to_plot["all_100_t2690_eteq_tol_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eteq_tol_10/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eteq_tol_11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eteq_tol_11/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eteq_tol_12"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eteq_tol_12/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eteq_tol_8"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eteq_tol_8/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eteq_tol_9"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eteq_tol_9/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eteq_tol_eq"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eteq_tol_eq/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eth_tol_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eth_tol_10/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eth_tol_11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eth_tol_11/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eth_tol_12"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eth_tol_12/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eth_tol_8"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eth_tol_8/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eth_tol_9"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eth_tol_9/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eth_tol_eq"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eth_tol_eq/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_etl_tol_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_etl_tol_10/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_etl_tol_11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_etl_tol_11/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_etl_tol_8"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_etl_tol_8/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_etl_tol_9"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_etl_tol_9/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_etl_tol_eq"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_etl_tol_eq/Lev3_A?/Run/"

# runs_to_plot["t6115_tol11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol11/Lev3_A?/Run/"
# runs_to_plot["t6115_tol10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol10/Lev3_A?/Run/"
# runs_to_plot["t6115_tol9"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol9/Lev3_A?/Run/"
# runs_to_plot["t6115_tol8"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol8/Lev3_A?/Run/"
# runs_to_plot["t6115_tol7"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol7/Lev3_A?/Run/"
# runs_to_plot["t6115_tol11_AMR"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol11_AMR/Lev3_A?/Run/"
# runs_to_plot["t6115_tol10_AMR"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol10_AMR/Lev3_A?/Run/"
# runs_to_plot["t6115_tol9_AMR"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol9_AMR/Lev3_A?/Run/"
# runs_to_plot["t6115_tol8_AMR"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol8_AMR/Lev3_A?/Run/"
# runs_to_plot["t6115_tol7_AMR"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol7_AMR/Lev3_A?/Run/"
# runs_to_plot["t6115_tol8_linf"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol8_linf/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.02"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.02/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.03"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.03/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.041"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.041/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.042"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.042/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.043"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.043/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.044"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.044/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.045"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.045/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.046"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.046/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.047"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.047/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.048"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.048/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.049"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.049/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.050"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.050/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.052"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.052/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.054"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.054/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.056"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.056/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_2.368e-07"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_2.368e-07/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_1.692e-07"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_1.692e-07/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_1.015e-07"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_1.015e-07/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_6.767e-08"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_6.767e-08/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_5.075e-08"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_5.075e-08/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_3.383e-08"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_3.383e-08/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_2.256e-08"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_2.256e-08/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_1.692e-08"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_1.692e-08/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_1.128e-08"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_1.128e-08/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_6.767e-09"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_6.767e-09/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_4.833e-09"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_4.833e-09/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_3.383e-09"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_3.383e-09/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_1.692e-09"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_1.692e-09/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_1.128e-09"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_1.128e-09/Lev3_A?/Run/"


# runs_to_plot["all_100_t2710_0.021_0.021"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2710_0.021_0.21/Lev3_AE/Run/"
# runs_to_plot["all_100_t2710_0.021_0.022"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2710_0.021_0.22/Lev3_AE/Run/"
# runs_to_plot["all_100_t2710_0.021_0.0225"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2710_0.021_0.225/Lev3_AE/Run/"
# runs_to_plot["all_100_t2710_0.021_0.023"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2710_0.021_0.23/Lev3_AE/Run/"
# runs_to_plot["all_100_t2710_0.021_0.024"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2710_0.021_0.24/Lev3_AE/Run/"
# runs_to_plot["all_100_t2710_0.021_max_tol8"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2710_0.021_max_tol8/Lev3_AE/Run/"
# runs_to_plot["all_100_t2710_0.021_max_tol9"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2710_0.021_max_tol9/Lev3_AE/Run/"
# runs_to_plot["all_100_t2710_0.021_max_tol10"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2710_0.021_max_tol10/Lev3_AE/Run/"
# runs_to_plot["all_100_t2710_0.021_max_tol10.5"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2710_0.021_max_tol10.5/Lev3_AE/Run/"
# runs_to_plot["all_100_t2710_0.021_max_tol11"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2710_0.021_max_tol11/Lev3_AE/Run/"

# runs_to_plot["eq_t4000_tol10"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_all_10/eq_t4000_tol10/Lev3_A?/Run/"
# runs_to_plot["eq_t4000_tol5_10"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_all_10/eq_t4000_tol5_10/Lev3_A?/Run/"
# runs_to_plot["eq_t4000_tol5_11"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_all_10/eq_t4000_tol5_11/Lev3_A?/Run/"
# runs_to_plot["eq_t4000_tol9"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_all_10/eq_t4000_tol9/Lev3_A?/Run/"
# runs_to_plot["t4000_tol10"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_all_10/t4000_tol10/Lev3_A?/Run/"
# runs_to_plot["t4000_tol5_10"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_all_10/t4000_tol5_10/Lev3_A?/Run/"
# runs_to_plot["t4000_tol5_11"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_all_10/t4000_tol5_11/Lev3_A?/Run/"
# runs_to_plot["t4000_tol8"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_all_10/t4000_tol8/Lev3_A?/Run/"
# runs_to_plot["t4000_tol9"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_all_10/t4000_tol9/Lev3_A?/Run/"

# runs_to_plot["Lev3_AA_tol10_all_10"]  = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/rd_all_10_tol11/Lev3_AA_tol10_all_10/Lev3_A?_/Run/"
# runs_to_plot["Lev3_AA_tol11_all_10"]  = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/rd_all_10_tol11/Lev3_AA_tol11_all_10/Lev3_A?_/Run/"
# runs_to_plot["Lev3_AA_tol12_all_10"]  = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/rd_all_10_tol11/Lev3_AA_tol12_all_10/Lev3_A?_/Run/"
# runs_to_plot["Lev3_AA_tol5_10_all_10"]  = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/rd_all_10_tol11/Lev3_AA_tol5_10_all_10/Lev3_A?_/Run/"
# runs_to_plot["Lev3_AA_tol5_11_all_10"]  = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/rd_all_10_tol11/Lev3_AA_tol5_11_all_10/Lev3_A?_/Run/"
# runs_to_plot["Lev3_AA_tol10"]  = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/rd_all_10_tol11/Lev3_AA_tol10/Lev3_A?_/Run/"
# runs_to_plot["Lev3_AA_tol11"]  = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/rd_all_10_tol11/Lev3_AA_tol11/Lev3_A?_/Run/"
# runs_to_plot["Lev3_AA_tol12"]  = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/rd_all_10_tol11/Lev3_AA_tol12/Lev3_A?_/Run/"
# runs_to_plot["Lev3_AA_tol5_10"]  = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/rd_all_10_tol11/Lev3_AA_tol5_10/Lev3_A?_/Run/"
# runs_to_plot["Lev3_AA_tol5_11"]  = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/rd_all_10_tol11/Lev3_AA_tol5_11/Lev3_A?_/Run/"

# runs_to_plot["3_100"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/changing_spectral_grid/3_100/Lev3_A?/Run/"
# runs_to_plot["3_10"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/changing_spectral_grid/3_10/Lev3_A?/Run/"
# runs_to_plot["3_1"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/changing_spectral_grid/3_1/Lev3_A?/Run/"
# runs_to_plot["2_100"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/changing_spectral_grid/2_100/Lev3_A?/Run/"
# runs_to_plot["2_10"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/changing_spectral_grid/2_10/Lev3_A?/Run/"
# runs_to_plot["2_1"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/changing_spectral_grid/2_1/Lev3_A?/Run/"
# runs_to_plot["1_100"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/changing_spectral_grid/1_100/Lev3_A?/Run/"
# runs_to_plot["1_10"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/changing_spectral_grid/1_10/Lev3_A?/Run/"

data_file_path = "extracted-FilterDiagnostics/BoundaryFilters.dir/ExpChebCoef.dir/SliceLFF.SphereA0.dat"
data_file_path = "extracted-PowerDiagnostics/SphereA0.dir/Bf0I1_NumberOfPiledUpModes.dat"
data_file_path = "extracted-PowerDiagnostics/SphereA0.dir/Bf0I1_PowerInHighestUnfilteredModes.dat"
data_file_path = "extracted-PowerDiagnostics/SphereA0.dir/Bf0I1_TruncationError.dat"
data_file_path = "extracted-PowerDiagnostics/SphereA0.dir/Bf1S2_TruncationError.dat"
# data_file_path = "extracted-RhsExpense/CostPerProc.dir/Proc00.dat"
# data_file_path = "extracted-RhsExpense/CostPerSubdomain.dir/SphereB2.dat"
# data_file_path = "extracted-OrbitDiagnostics/OrbitalPhase.dat"
# data_file_path = "extracted-AdjustGridExtents/SphereA0.dir/Extents.dat"
# data_file_path = "extracted-ControlNthDeriv/ExpansionFactor.dir/a.dat"
# data_file_path = "extracted-ControlNthDeriv/Trans.dir/Tx.dat"
# data_file_path = "extracted-FilterDiagnostics/SubdomainFilters.dir/ExpChebCoef.dir/SphereB0.dat"

column_names, runs_data_dict = load_data_from_levs(runs_to_plot,data_file_path)
print(column_names)

In [ ]:
moving_avg_len=0
save_path = None
diff_base = None
x_axis = 't(M)'

# diff_base = 'Lev3_AA_tol12_all_10'
# add_max_and_min_val(runs_data_dict)
# y_axis = 'max_val'
# y_axis = 'min_val'

y_axis = 'GridDiagPowerpsi'
y_axis = 'GridDiagPowerkappa'
# y_axis = 'CostPerPoint'
# y_axis = 'Cost'
# y_axis = 'NumberOfPoints'
# y_axis = 'Q'
# y_axis = 'lambda'
# y_axis = 'Bf0I1'

minT = 0
minT = 2100
# minT = 2710

maxT = 4000
# maxT = 9300
# maxT = 9375
# maxT = 40000
# moving_avg_len = 50
# moving_avg_len = 10

plot_fun = lambda x,y,label : plt.plot(x,y,label=label)
# plot_fun = lambda x,y,label : plt.plot(x,y,label=label,marker='x')
# plot_fun = lambda x,y,label : plt.semilogy(x,y,label=label)
# plot_fun = lambda x,y,label : plt.semilogy(x,y,label=label,marker='x')
# plot_fun = lambda x,y,label : plt.loglog(x,y,label=label) 
# plot_fun = lambda x,y,label : plt.scatter(x,y,label=label,s=10,marker="x")
# save_path = "/groups/sxs/hchaudha/rough/high_acc_plots/"
# save_path = "/groups/sxs/hchaudha/rough/plots/"
# save_path = "/home/hchaudha/notes/spec_accuracy/figures/"
legend_dict = {}
for key in runs_data_dict.keys():
  legend_dict[key] = None

# legend_dict = { '3_DH_q1_ns_d18_L3': "Lev3",
#                 '3_DH_q1_ns_d18_L6': "Lev6",
#                 'all_10': "Lev3_all_tols_10",
#                 'all_100': "Lev3_all_tols_100",
#                 'near_bhs_10': "Lev3_sphere_AB_tols_10",
#                 'near_bhs_100': "Lev3_sphere_AB_tols_100"}
# legend_dict = {
#  '3_DH_q1_ns_d18_L3':"Lev3_ode_tol_1e-8",
#  '3_DH_q1_ns_d18_L3_tol9':"Lev3_ode_tol_1e-9",
#  '3_DH_q1_ns_d18_L3_tol10':"Lev3_ode_tol_1e-10",
#  '3_DH_q1_ns_d18_L3_tol11':"Lev3_ode_tol_1e-11",
#  '3_DH_q1_ns_d18_L3_all_100_tol10':"Lev3_AMR_tol_100_ode_tol_1e-11",
#  }

with plt.style.context('default'):
  plt.rcParams["figure.figsize"] = (12,10)
  plt.rcParams["figure.autolayout"] = True
  plot_graph_for_runs(runs_data_dict, x_axis, y_axis, minT, maxT, legend_dict=legend_dict, save_path=save_path, moving_avg_len=moving_avg_len, plot_fun=plot_fun, diff_base=diff_base)
  plt.show()

# Plots for one

In [ ]:
main_path = Path("/groups/sxs/hchaudha/spec_runs/")
run_path = main_path/Path("3_DH_q1_ns_d18_L3/Ev/Lev3_A?/Run/")
# run_path = main_path/Path("3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs/Lev3_AD/Run/")
# run_path = main_path/Path("3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs/Lev3_AD/Run/")
# run_path = main_path/Path("3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_tol_9/Lev3_AD/Run/")
# run_path = main_path/Path("3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_tol_10/Lev3_AD/Run/")
# run_path = main_path/Path("3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_tol_11/Lev3_AD/Run/")

data_files = {}

# data_files["GrAdjustSubChunksToDampingTimes"] = {"path":run_path/"GrAdjustSubChunksToDampingTimes.dat", "prefix":None}
# data_files["MemoryInfo"] = {"path":run_path/"MemoryInfo.dat", "prefix":None}
# data_files["MinimumGridSpacing"] = {"path":run_path/"MinimumGridSpacing.dat", "prefix":None}
# data_files["DiagInclinationAngle"] = {"path":run_path/"DiagInclinationAngle.dat", "prefix":None}
# data_files["ApparentHorizons/Trajectory_AhA"] = {"path":run_path/"ApparentHorizons/Trajectory_AhA.dat", "prefix":"AhA"}
# data_files["ApparentHorizons/MinCharSpeedAhA"] = {"path":run_path/"ApparentHorizons/MinCharSpeedAhA.dat", "prefix":"AhA"}
# data_files["ApparentHorizons/Trajectory_AhB"] = {"path":run_path/"ApparentHorizons/Trajectory_AhB.dat", "prefix":"AhB"}
# data_files["ApparentHorizons/SmoothCoordSepHorizon"] = {"path":run_path/"ApparentHorizons/SmoothCoordSepHorizon.dat", "prefix":None}
# data_files["ApparentHorizons/MinCharSpeedAhB"] = {"path":run_path/"ApparentHorizons/MinCharSpeedAhB.dat", "prefix":"AhB"}
# data_files["ApparentHorizons/RescaledRadAhB"] = {"path":run_path/"ApparentHorizons/RescaledRadAhB.dat", "prefix":"AhB"}
# data_files["ApparentHorizons/AhACoefs"] = {"path":run_path/"ApparentHorizons/AhACoefs.dat", "prefix":"AhA"}
# data_files["ApparentHorizons/AhB"] = {"path":run_path/"ApparentHorizons/AhB.dat", "prefix":"AhB"}
# data_files["ApparentHorizons/HorizonSepMeasures"] = {"path":run_path/"ApparentHorizons/HorizonSepMeasures.dat", "prefix":None}
# data_files["ApparentHorizons/AhA"] = {"path":run_path/"ApparentHorizons/AhA.dat", "prefix":"AhA"}
# data_files["ApparentHorizons/RescaledRadAhA"] = {"path":run_path/"ApparentHorizons/RescaledRadAhA.dat", "prefix":"AhA"}
# data_files["ApparentHorizons/AhBCoefs"] = {"path":run_path/"ApparentHorizons/AhBCoefs.dat", "prefix":"AhB"}
# data_files["TimeInfo"] = {"path":run_path/"TimeInfo.dat", "prefix":None}
# data_files["GrAdjustMaxTstepToDampingTimes"] = {"path":run_path/"GrAdjustMaxTstepToDampingTimes.dat", "prefix":None}
# data_files["FailedTStepperDiag"] = {"path":run_path/"FailedTStepperDiag.dat", "prefix":None}
# data_files["DiagAhSpeedA"] = {"path":run_path/"DiagAhSpeedA.dat", "prefix":"AhA"}
# data_files["DiagAhSpeedB"] = {"path":run_path/"DiagAhSpeedB.dat", "prefix":"AhB"}
# data_files["CharSpeedNorms/CharSpeeds_Max_SliceLFF.SphereA0"] = {"path":run_path/"CharSpeedNorms/CharSpeeds_Max_SliceLFF.SphereA0.dat", "prefix":"Max_A0"}
# data_files["CharSpeedNorms/CharSpeeds_Min_SliceLFF.SphereA0"] = {"path":run_path/"CharSpeedNorms/CharSpeeds_Min_SliceLFF.SphereA0.dat", "prefix":"Min_A0"}
# data_files["CharSpeedNorms/CharSpeeds_Min_SliceLFF.SphereB0"] = {"path":run_path/"CharSpeedNorms/CharSpeeds_Min_SliceLFF.SphereB0.dat", "prefix":"Max_B0"}
# data_files["CharSpeedNorms/CharSpeeds_Max_SliceLFF.SphereB0"] = {"path":run_path/"CharSpeedNorms/CharSpeeds_Max_SliceLFF.SphereB0.dat", "prefix":"Min_B0"}
# data_files["CharSpeedNorms/CharSpeeds_Max_SliceUFF.SphereC29"] = {"path":run_path/"CharSpeedNorms/CharSpeeds_Max_SliceUFF.SphereC29.dat", "prefix":"Max_C29"}
# data_files["CharSpeedNorms/CharSpeeds_Min_SliceUFF.SphereC29"] = {"path":run_path/"CharSpeedNorms/CharSpeeds_Min_SliceUFF.SphereC29.dat", "prefix":"Min_C29"}
# data_files["ConstraintNorms/NormalizedGhCe_Norms"] = {"path":run_path/"ConstraintNorms/NormalizedGhCe_Norms.dat", "prefix":None}
# data_files["ConstraintNorms/GhCeExt_Norms"] = {"path":run_path/"ConstraintNorms/GhCeExt_Norms.dat", "prefix":None}
# data_files["ConstraintNorms/GhCe_L2"] = {"path":run_path/"ConstraintNorms/GhCe_L2.dat", "prefix":None}
# data_files["ConstraintNorms/GhCeExt_L2"] = {"path":run_path/"ConstraintNorms/GhCeExt_L2.dat", "prefix":None}
# data_files["ConstraintNorms/GhCeExt"] = {"path":run_path/"ConstraintNorms/GhCeExt.dat", "prefix":None}
# data_files["ConstraintNorms/GhCe"] = {"path":run_path/"ConstraintNorms/GhCe.dat", "prefix":None}
# data_files["ConstraintNorms/GhCe_VolL2"] = {"path":run_path/"ConstraintNorms/GhCe_VolL2.dat", "prefix":None}
data_files["ConstraintNorms/GhCe_Norms"] = {"path":run_path/"ConstraintNorms/GhCe_Norms.dat", "prefix":None}
# data_files["ConstraintNorms/NormalizedGhCe_Linf"] = {"path":run_path/"ConstraintNorms/NormalizedGhCe_Linf.dat", "prefix":None}
# data_files["ConstraintNorms/GhCe_Linf"] = {"path":run_path/"ConstraintNorms/GhCe_Linf.dat", "prefix":None}
data_files["TStepperDiag"] = {"path":run_path/"TStepperDiag.dat", "prefix":None}
# data_files["DiagCutXCorrection"] = {"path":run_path/"DiagCutXCorrection.dat", "prefix":None}


# data = read_dat_file_across_AA(str(data_files['TimeInfo']['path']))
for key in data_files:
  data_files[key]["dataframe"] = read_dat_file_across_AA(str(data_files[key]['path']))
  cols = list(data_files[key]["dataframe"].columns)
  # Make new cols names s.t. the first cols is 't' and add prefix as required
  new_cols = []
  new_cols.append('t')
  if data_files[key]['prefix'] is None:
    [new_cols.append(name) for name in cols[1:]]
  else:
    [new_cols.append(name+"_"+data_files[key]['prefix']) for name in cols[1:]]
  data_files[key]["dataframe"].columns = new_cols

  # Set 't' to be a index and copy it into a column called 'time'
  # data_files[key]["dataframe"]["time"] = data_files[key]["dataframe"]["t"]
  # data_files[key]["dataframe"].set_index('t', inplace=True)

combined = None
for key in data_files:
  if combined is None:
    combined = data_files[key]["dataframe"]
    continue
  else:
    combined  = pd.merge(combined,data_files[key]["dataframe"],on='t',how='outer')


In [ ]:
combined = combined.sort_values(by='t')
for i in combined.columns:
  print(i)

In [ ]:
# plt.scatter(combined['t'],combined['SuggestedDampingTime'])

x_val = 't'
plot_list=[
  # ('Linf(GhCe) on SphereA0','semilogy',None,None,'x'),
  ('Linf(GhCe)','semilogy',None,None,'.'),
  ('dt','plot',1e-3,None,None),
]

for i in plot_list:
  y_val, plot_type, mul_factor, add_factor, marker = i

  label = y_val
  if mul_factor is not None:
    label = f"{label}*{mul_factor}"
  else:
    mul_factor = 1
  if add_factor is not None:
    label = f"{label}+{add_factor}"
  else:
    add_factor = 0


  match plot_type:
    case 'semilogy':
      plt.semilogy(combined[x_val],combined[y_val]*mul_factor+add_factor,marker=marker,label=label)
    case 'plot':
      plt.plot(combined[x_val],combined[y_val]*mul_factor+add_factor,marker=marker,label=label)

title = str(run_path).split('/')[-4]
save_name = str(run_path).split('/')[-4]
for i in str(run_path).split('/')[-3:-1]:
  title = title +"/" +i
  save_name = save_name+"&"+i


plt.title(title)
plt.xlabel(x_val)
plt.legend()
plt.tight_layout()
plt.savefig(f"/groups/sxs/hchaudha/rough/plots/{save_name}.png",dpi=500)
plt.show()

In [ ]:
import pandas as pd
def check_duplicate_rows(df, subset_column):
    """
    Function to find and print duplicate rows in a DataFrame based on a specified column.
    It checks if all duplicate rows are identical or if they differ in some columns.

    Parameters:
    df (pd.DataFrame): The DataFrame to check for duplicates.
    subset_column (str): The column to check for duplicate values.

    Returns:
    None
    """
    # Get rows with duplicate values in the subset column
    duplicate_mask = df.duplicated(subset=[subset_column], keep=False)
    duplicate_rows = df[duplicate_mask]
    
    # Sort the duplicate rows by the subset column to group them together
    duplicate_rows_sorted = duplicate_rows.sort_values(by=subset_column)
    
    # Iterate through groups of rows with the same value in the subset column
    for _, group in duplicate_rows_sorted.groupby(subset_column):
        if len(group) > 1:
            print(f"\nRows with '{subset_column}' value: {group[subset_column].iloc[0]}")
            
            # Check if all rows in the group are identical
            identical = all(group.iloc[0].equals(row) for _, row in group.iterrows())
            if identical:
                print("All rows are identical:")
                print(group)
            else:
                print("Rows differ in some columns:")
                print(group)
                
                # Optionally, show which columns differ
                differing_columns = group.columns[group.nunique() > 1].tolist()
                print(f"Columns with differences: {differing_columns}")



# Usage example
# combined = pd.read_csv('your_data.csv')  # Load your data
# check_duplicate_rows(combined, 't')

In [ ]:
check_duplicate_rows(combined,'t')

In [ ]:
import pandas as pd

def check_duplicate_rows(df, subset_column):
    # Get rows with duplicate values in the subset column
    duplicate_mask = df.duplicated(subset=[subset_column], keep=False)
    duplicate_rows = df[duplicate_mask]
    
    # Sort the duplicate rows by the subset column to group them together
    duplicate_rows_sorted = duplicate_rows.sort_values(by=subset_column)
    
    # Iterate through groups of rows with the same value in the subset column
    for _, group in duplicate_rows_sorted.groupby(subset_column):
        if len(group) > 1:
            print(f"\nRows with '{subset_column}' value: {group[subset_column].iloc[0]}")
            
            # Check if all rows in the group are identical
            if group.drop_duplicates().shape[0] == 1:
                print("All rows are identical:")
                print(group)
            else:
                print("Rows differ in some columns:")
                print(group)
                
                # Show which columns differ
                differing_columns = group.columns[group.nunique() > 1].tolist()
                print(f"Columns with differences: {differing_columns}")
                
                # Show the differences
                for col in differing_columns:
                    if col != subset_column:
                        print(f"\nDifferences in column '{col}':")
                        print(group[['t', col]])

# Usage example
# combined = pd.read_csv('your_data.csv')  # Load your data
check_duplicate_rows(combined, 't')

In [ ]:
combined = combined.sort_values(by='t')

In [ ]:
combined['MinCharSpeedAhA[7]_AhA']

In [ ]:
col_names = []
for i in data:
  print(i.columns.is_unique)
  col_names = col_names+list(i.columns)

In [ ]:
len(col_names), len(set(col_names))

In [ ]:
pd.concat(data,axis=1)

# Linf plots

In [ ]:
runs_to_plot = {}
runs_to_plot["t6115_tol8_linf"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol8_linf/Lev3_A?/Run/"

data_file_path="ConstraintNorms/Linf.dat"


column_names_linf, runs_data_dict_linf = load_data_from_levs(runs_to_plot,data_file_path)
print(column_names_linf)
print(runs_data_dict_linf.keys())

In [ ]:
data = runs_data_dict_linf['t6115_tol8_linf']

In [ ]:
domain_list = []
constraint_list = []
component_list = []

# domain_list.append("SphereB0")
domain_list.append("erMA0")

# constraint_list.append('1Con')
constraint_list.append('2Con')
# constraint_list.append('3Con')

component_list.append('t')

temp_list = copy.copy(column_names_linf)
col_domains = []
for col in temp_list:
  for domain in domain_list:
    if domain in col:
      col_domains.append(col)

temp_list = col_domains
col_domains = []
for col in temp_list:
  for constraint in constraint_list:
    if constraint in col:
      col_domains.append(col)

if len(component_list) > 0:
  temp_list = col_domains
  col_domains = []
  for col in temp_list:
    for component in component_list:
      if component in col:
        col_domains.append(col)

col_domains

In [ ]:
moving_avg_len = None

x = 'time'
# moving_avg_len = 50*3


for col in col_domains:
  if moving_avg_len is not None:
    plt.semilogy(np.array(data[x])[moving_avg_len-1:],moving_average_valid(data[col],moving_avg_len),label=col)
  else:
    plt.semilogy(data[x],data[col],label=col)
  # plt.plot(data[x],data[col],label=col)
plt.xlabel(x)
plt.legend()
plt.show()

In [ ]:
len(moving_average_valid(data[col],average_over))

# Plot horizons.h5

In [ ]:
current_dir = os.getcwd()
if "hchaudha" in current_dir:
    base_path = Path("/groups/sxs/hchaudha/spec_runs")
else:
    base_path = Path("/panfs/ds09/sxs/himanshu/gauge_stuff/gauge_driver_runs/runs")

In [ ]:
runs_to_plot = {}
runs_to_plot["all_100"] =  "3_DH_q1_ns_d18_L3_higher_acc/all_100/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
data_dict = load_horizon_data_from_levs(base_path, runs_to_plot)
data_dict = flatten_dict(data_dict)
data_dict[list(data_dict.keys())[0]].columns

In [ ]:
moving_avg_len = 0
save_path = None

x_axis = 't'
# y_axis = 'ArealMass'
# y_axis = 'ChristodoulouMass'
# y_axis = 'CoordCenterInertial_0'
# y_axis = 'CoordCenterInertial_1'
# y_axis = 'CoordCenterInertial_2'
# y_axis = 'DimensionfulInertialSpin_0'
# y_axis = 'DimensionfulInertialSpin_1'
# y_axis = 'DimensionfulInertialSpin_2'
# y_axis = 'DimensionfulInertialCoordSpin_0'
# y_axis = 'DimensionfulInertialCoordSpin_1'
# y_axis = 'DimensionfulInertialCoordSpin_2'
# y_axis = 'DimensionfulInertialSpinMag'
# y_axis = 'SpinFromShape_0'
# y_axis = 'SpinFromShape_1'
# y_axis = 'SpinFromShape_2'
# y_axis = 'SpinFromShape_3'
# y_axis = 'chiInertial_0'
# y_axis = 'chiInertial_1'
# y_axis = 'chiInertial_2'
# y_axis = 'chiMagInertial'



# moving_avg_len=25
minT = 2271
maxT = 8000

plot_fun = lambda x,y,label : plt.plot(x,y,label=label)
# plot_fun = lambda x,y,label : plt.semilogy(x,y,label=label)
# plot_fun = lambda x,y,label : plt.loglog(x,y,label=label)
# plot_fun = lambda x,y,label : plt.scatter(x,y,label=label)
# save_path = "/panfs/ds09/sxs/himanshu/scripts/report/not_tracked/temp2/"

filtered_dict = {}
allowed_horizons = ["AhB"]
for horizons in allowed_horizons:
  for runs_keys in data_dict.keys():
    if horizons in runs_keys:
      filtered_dict[runs_keys] = data_dict[runs_keys]
 
with plt.style.context('default'):
  plt.rcParams["figure.figsize"] = (12,10)
  plt.rcParams["figure.figsize"] = (8,6)
  plt.rcParams["figure.autolayout"] = True
  plot_graph_for_runs(filtered_dict, x_axis, y_axis, minT, maxT, save_path=save_path, moving_avg_len=moving_avg_len, plot_fun=plot_fun)

plt.show()

In [ ]:
bh = 'corrected_coord_spin2_AhB'
y_axis1 = 'chiInertial_0'
y_axis2 = 'CoordSpinChiInertial_0'

X = data_dict[bh][x_axis]
Y1 = data_dict[bh][y_axis1]
Y2 = data_dict[bh][y_axis2]
plt.plot(X,Y1,label=y_axis1)
plt.plot(X,Y2,label=y_axis2)
plt.xlabel(x_axis)
# plt.ylabel(y_axis1+" - "+y_axis2)
plt.legend()
# plt.title()
plt.show()

In [ ]:
filtered_dict.keys()

In [ ]:
x_axis = 't'
y_axis = 'ChristodoulouMass'
minT = 500
maxT = 800
run1 = filtered_dict['AccTest_q1ns_Lev5_AhA']
# run1 = filtered_dict['AccTest_q1ns_Lev6p_AhA']
run2 = filtered_dict['AccTest_q1ns_Lev6p_AhA']
interp_grid_pts = run1[x_axis].size

In [ ]:
interp_run1 = CubicSpline(run1[x_axis].to_numpy(),run1[y_axis].to_numpy())
interp_run2 = CubicSpline(run2[x_axis].to_numpy(),run2[y_axis].to_numpy())
interp_grid = np.arange(minT,maxT,(maxT-minT)/interp_grid_pts)

plt.plot(interp_grid, interp_run2(interp_grid) - interp_run1(interp_grid))
plt.xlabel(x_axis)
plt.ylabel(y_axis)
plt.legend()

In [ ]:
plt.plot(interp_grid, interp_run2(interp_grid) - interp_run1(interp_grid))

In [ ]:
def inertial_dist(run_name:str, data_dict):
    ct = data_dict[f"{run_name}_AhA"].t
    dx = data_dict[f"{run_name}_AhA"].CoordCenterInertial_0 - data_dict[f"{run_name}_AhB"].CoordCenterInertial_0
    dy = data_dict[f"{run_name}_AhA"].CoordCenterInertial_1 - data_dict[f"{run_name}_AhB"].CoordCenterInertial_1
    dz = data_dict[f"{run_name}_AhA"].CoordCenterInertial_2 - data_dict[f"{run_name}_AhB"].CoordCenterInertial_2

    dx = np.sqrt(dx**2 + dy**2 + dz**2)

    return ct,dx


In [ ]:
for run_name in runs_to_plot.keys():
    ct,dx = inertial_dist(run_name,data_dict)
    plt.plot(ct,dx,label=run_name)
    plt.legend()

In [ ]:
print(data_dict.keys())
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(data_dict['76_ngd_master_mr1_50_3000_AhA'].describe())

# Combine all paraview files into a single file

In [ ]:
base_folder = Path("/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/near_bhs_100_obs/")

Lev = 3
file_pattern =f"Lev{Lev}_A[A-Z]/Run/GaugeVis.pvd"
file_patternGrid =f"Lev{Lev}_A[A-Z]/Run/GaugeVisGrid.pvd"
file_patternAll =f"Lev{Lev}_A[A-Z]/Run/GuageVisAll.pvd"

combine_pvd_files(base_folder,file_pattern)
combine_pvd_files(base_folder,file_patternGrid)
combine_pvd_files(base_folder,file_patternAll)

```Python
# Create GaugeVis
command = f"cd {base_folder} && mkdir ./GaugeVis"
status = subprocess.run(command, capture_output=True, shell=True, text=True)
if status.returncode == 0:
  print(f"Succesfully created GaugeVis in {base_folder}")
else:
  sys.exit(
      f"GaugeVis creation failed in {base_folder} with error: \n {status.stderr}")

# Create GaugeVis subfolder
vtu_folder_path = base_folder+"/GaugeVis/GaugeVis"
command = f"mkdir {vtu_folder_path}"
status = subprocess.run(command, capture_output=True, shell=True, text=True)
if status.returncode == 0:
  print(f"Succesfully created {vtu_folder_path}")
else:
  sys.exit(
      f"GaugeVis creation failed as {vtu_folder_path} with error: \n {status.stderr}")


# Copy vtu files
GaugeVisFolder=[]

for paths in path_collection:
  GaugeVisFolder.append(paths[:-4])

for paths in GaugeVisFolder:
  command = f"cp {paths}/*.vtu {vtu_folder_path}/"
  status = subprocess.run(command, capture_output=True, shell=True, text=True)
  if status.returncode == 0:
    print(f"Succesfully copied vtu files from {paths}")
  else:
    sys.exit(
        f"Copying vtu files from {paths} failed with error: \n {status.stderr}")

```

# Profiler results

# Make report (do not run randomly)

In [ ]:
with open("./report_new_gauge.json") as report_data:
  data = json.load(report_data)

os.mkdir(data['report_folder'])

subfolders = []
for folders in data['runs_to_track']:
  subfolders_path = data['report_folder'] + "/" + path_to_folder_name(folders) + "/"
  print(subfolders_path)
  os.mkdir(subfolders_path)
  subfolders.append(subfolders_path)

In [ ]:
runs_still_going_on = True
while runs_still_going_on:
  # time.sleep(data['report_generation_frequency'])

  for i,run_folder_path in enumerate(data['runs_to_track']):
    # if is_the_current_run_going_on(run_folder_path) or True:
    if True:
      plots_for_a_folder(data['things_to_plot'],subfolders[i],run_folder_path)
    print(run_folder_path)


  runs_still_going_on = False
  print("all done")


### Save all columns and data files paths

In [ ]:
# Write all the cols in the dat files for reference
lev_golb="/panfs/ds09/sxs/himanshu/gauge_stuff/gauge_driver_runs/runs/gauge_driver_kerr_target_50_50_0_16_16_01/Ev/Lev1_AA"
dat_files_glob=lev_golb+"/Run/**/**.dat"
path_pattern = dat_files_glob

path_collection = []
for folder_name in glob.iglob(path_pattern, recursive=True):
    if os.path.isdir(folder_name) or os.path.isfile(folder_name):
        path_collection.append(folder_name)
        print(folder_name.split("/")[-1])


column_data_for_dat_files = {
  'columns_of_dat_files' : [
  ] 
}

for file_path in path_collection:
  file_name = file_path.split("/")[-1]
  columns_list =  list(read_dat_file(file_path).columns)
  column_data_for_dat_files['columns_of_dat_files'].append({
    'file_name': file_name,
    'file_path': file_path,
    'columns': columns_list
  })


with open('./column_data_for_dat_files.json', 'w') as outfile:
  json.dump(column_data_for_dat_files, outfile, indent=2)

In [ ]:
def JoinH5(h5_file_list, output_path, output_file_name):

  file_list_to_str = ""
  for h5file in h5_file_list:
    file_list_to_str += h5file + " "

  command = f"cd {output_path} && {spec_home}/Support/bin/JoinH5 -o {output_file_name} {file_list_to_str}"
  status = subprocess.run(command, capture_output=True, shell=True, text=True)
  if status.returncode == 0:
    print(f"Succesfully ran JoinH5 in {output_path}")
  else:
    sys.exit(
        f"JoinH5 failed in {output_path} with error: \n {status.stderr}")


def ExtractFromH5(h5_file, output_path):

  command = f"cd {output_path} && {spec_home}/Support/bin/ExtractFromH5 {h5_file}"
  status = subprocess.run(command, capture_output=True, shell=True, text=True)
  if status.returncode == 0:
    print(f"Succesfully ran ExtractFromH5 in {output_path}")
  else:
    sys.exit(
        f"ExtractFromH5 failed in {output_path} with error: \n {status.stderr}")



In [ ]:
output_base_path= "/panfs/ds09/sxs/himanshu/gauge_stuff/gauge_driver_runs/runs/profiler_results"


base_folder = "/panfs/ds09/sxs/himanshu/gauge_stuff/gauge_driver_runs/runs/49_ngd_weird_gauge_mr1"
file_pattern = base_folder+"/Ev/Lev1_A?/Run/Profiler.h5"

path_pattern = file_pattern
path_collection = []

# make a folder in the output directory
save_folder = output_base_path+"/"+base_folder.split("/")[-1]
os.mkdir(save_folder)


# Find all the files that match the required pattern of the file
for folder_name in glob.iglob(path_pattern, recursive=True):
    if os.path.isdir(folder_name) or os.path.isfile(folder_name):
        path_collection.append(folder_name)
        print(folder_name)

JoinH5(path_collection,save_folder,"Profiler.h5")
ExtractFromH5("Profiler.h5",save_folder)

# Save path of all the summary files in extracted data

file_pattern = base_folder+"/Ev/Lev1_A?/Run/Profiler.h5"

path_pattern = file_pattern
path_collection = []

# Find all the files that match the required pattern of the file
for folder_name in glob.iglob(path_pattern, recursive=True):
    if os.path.isdir(folder_name) or os.path.isfile(folder_name):
        path_collection.append(folder_name)
        print(folder_name)

In [ ]:
# Find all the Summary files 
summary_file_pattern = save_folder+"/**/Summary.txt"
summary_file_collection = []

for file_path in glob.iglob(summary_file_pattern, recursive=True):
    if os.path.isdir(file_path) or os.path.isfile(file_path):
        summary_file_collection.append(file_path)
        print(file_path)

summary_file_collection.sort()

In [ ]:
file_path = "/panfs/ds09/sxs/himanshu/gauge_stuff/gauge_driver_runs/runs/profiler_results/49_ngd_weird_gauge_mr1/extracted-Profiler/Step10522.dir/Summary.txt"



## AmrTolerances.input

In [ ]:
Lev=8
TruncationErrorMax = 0.000216536 * 4**(-Lev)
ProjectedConstraintsMax = 0.216536 * 4**(-Lev)
TruncationErrorMaxA = TruncationErrorMax*1.e-4
TruncationErrorMaxB = TruncationErrorMax*1.e-4

AhMaxRes  = TruncationErrorMax
AhMinRes  = AhMaxRes / 10.0

AhMaxTrunc=TruncationErrorMax
AhMinTrunc=AhMaxTrunc / 100.0

print(f"AhMinRes={AhMinRes};")
print(f"AhMaxRes={AhMaxRes};")
print(f"AhMinTrunc={AhMinTrunc};")
print(f"AhMaxTrunc={AhMaxTrunc};")
print(f"TruncationErrorMax={TruncationErrorMax};")
print(f"TruncationErrorMaxA={TruncationErrorMaxA};")
print(f"TruncationErrorMaxB={TruncationErrorMaxB};")
print(f"ProjectedConstraintsMax={ProjectedConstraintsMax};")

# Rough

In [ ]:
x = np.arange(100)/100
y1 = np.sin(x)
y2 = np.cos(x)

styles =  plt.style.available

for style in styles:
    print(style)
    plt.style.use(style)
    plt.plot(x,y1,label="y1asfasd")
    plt.plot(x,y2,label="y3asfasd")
    plt.title("asdf")
    plt.legend()
    plt.savefig(f"/panfs/ds09/sxs/himanshu/gauge_stuff/gauge_driver_runs/runs/make_report_del/{style}.png")
    plt.close()

In [ ]:
w = 4
print(np.convolve(np.arange(1,10),np.ones(w),'valid')/w)
print(np.arange(1,10))

In [ ]:
def moving_average(array,avg_len):
    return np.convolve(array,np.ones(avg_len))/avg_len

In [ ]:
def gauss(x,cen,scale):
  return np.exp(-(x-cen)**2/scale**2)

In [ ]:
d0 = 18
q = 1
d1 = 1/(1+q)*d0
d2 = q/(1+q)*d0
cen1 = d1
cen2 = -d2
scale1 = d2
scale2 = d1

tol1 = 3.38337e-10
tol2 = 3.38337e-10
tol_base = 3.38337e-6

d = 100
x_min,x_max = -d/2,d/2
X = np.linspace(x_min,x_max,1000)

fac_base = np.ones_like(X)
fac1 = gauss(X,cen1,scale1)
fac2 = gauss(X,cen2,scale2)

fac_total = fac_base+fac1+fac2
val = fac_base*np.log10(tol_base) + fac1*np.log10(tol1)+fac2*np.log10(tol2)
val = 10**(val/fac_total)
print(val.min(),val.max())

In [ ]:
f = plt.semilogy
f(X,val,label='tot')
# f(X,fac1*tol1,label='tol1')
# f(X,fac2*tol2,label='tol2')
# f(X,fac_base*tol_base,label='tol_base')
hor_lines = [val.min(),val[int(len(val)/2)],val.max()]
plt.hlines(y=hor_lines, xmin=-d/2, xmax=d/2, colors=['r', 'g', 'b'], linestyles='--', linewidth=2)
for y_value in hor_lines:
    plt.text(0, y_value*1.1, f'{y_value:.3e}', va='center', ha='left')


plt.ylabel('AMR tolerance')
plt.xlabel('x_axis')
plt.legend()
plt.show()

In [ ]:
plt.plot(X,gauss(X,0,50))
plt.show()

In [ ]:
def hist_files_to_dataframe(file_path):
  # Function to parse a single line and return a dictionary of values
  def parse_line(line):
      data = {}
      # Find all variable=value pairs
      pairs = re.findall(r'([^;=\s]+)=\s*([^;]+)', line)
      for var, val in pairs:
          # Hist-GrDomain.txt should be parsed a little differently
          if 'ResizeTheseSubdomains' in var:
              items = val.split('),')
              items[-1] = items[-1][:-1]
              for item in items:
                name,_,vals = item.split("(")
                r,l,m=vals[:-1].split(',')
                data[f"{name}_R"] = int(r)
                data[f"{name}_L"] = int(l)
                data[f"{name}_M"] = int(m)
          else:
              data[var] = float(val) if re.match(r'^[\d.e+-]+$', val) else val
      return data
  
  with open(file_path, 'r') as file:
    # Parse the lines
    data = []
    for line in file.readlines():
        data.append(parse_line(line.strip()))

    # Create a DataFrame
    df = pd.DataFrame(data)

  return df

hist_files_to_dataframe('/groups/sxs/hchaudha/spec_runs/2_SpKS_q1_sA_0_0_9_sB_0_0_9_d15/Ev/Lev3_AB/Run/Hist-GrDomain.txt')


In [ ]:
hist_files_to_dataframe('/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_etl_tol_10/Lev3_AD/Run_old/Hist-GrDomain.txt')


In [ ]:
hist_files_to_dataframe('/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_etl_tol_10/Lev3_AD/Run_old/Hist-GrDomain.txt')
# hist_files_to_dataframe('/groups/sxs/hchaudha/spec_runs/2_SpKS_q1_sA_0_0_9_sB_0_0_9_d15/Ev/Lev3_AB/Run/Hist-FuncLambdaFactorB.txt')

In [ ]:
import pandas as pd
import re

# Function to parse a single line and return a dictionary of values
def parse_line(line):
    data = {}
    # Find all variable=value pairs
    pairs = re.findall(r'([^;=]+)=\s*([\d.e+-]+)', line)
    for var, val in pairs:
        data[var] = float(val) if re.match(r'[\d.e+-]+', val) else val
    return data

# Read the file
with open('/groups/sxs/hchaudha/spec_runs/2_SpKS_q1_sA_0_0_9_sB_0_0_9_d15/Ev/Lev3_AB/Run/Hist-FuncLambdaFactorB.txt', 'r') as file:
    lines = file.readlines()

# Parse the lines
data = []
for line in lines:
    data.append(parse_line(line.strip()))

# Create a DataFrame
df = pd.DataFrame(data)

print(df)


In [ ]:
import pandas as pd
import re

# The input string
data = """SphereA0(Extents=(12,22,44)),SphereA1(Extents=(10,23,46)),SphereA2(Extents=(10,24,48)),SphereA3(Extents=(10,25,50)),SphereA4(Extents=(13,26,52)),SphereB0(Extents=(12,22,44)),SphereB1(Extents=(10,23,46)),SphereB2(Extents=(10,24,48)),SphereB3(Extents=(10,25,50)),SphereB4(Extents=(13,26,52)),SphereC0(Extents=(15,15,30)),SphereC1(Extents=(15,15,30)),SphereC2(Extents=(15,14,28)),SphereC3(Extents=(15,15,30)),SphereC4(Extents=(15,15,30)),SphereC5(Extents=(15,15,30)),SphereC6(Extents=(15,15,30)),SphereC7(Extents=(15,16,32)),SphereC8(Extents=(15,15,30)),SphereC9(Extents=(15,16,32)),SphereC10(Extents=(15,16,32)),SphereC11(Extents=(15,15,30)),SphereC12(Extents=(15,15,30)),SphereC13(Extents=(15,15,30)),SphereC14(Extents=(15,15,30)),SphereC15(Extents=(15,15,30)),SphereC16(Extents=(16,15,30)),SphereC17(Extents=(16,16,32)),SphereC18(Extents=(16,16,32)),SphereC19(Extents=(16,16,32)),SphereC20(Extents=(15,16,32)),SphereC21(Extents=(15,16,32)),SphereC22(Extents=(15,16,32)),SphereC23(Extents=(15,16,32)),SphereC24(Extents=(15,15,30)),SphereC25(Extents=(15,16,32)),SphereC26(Extents=(15,16,32)),SphereC27(Extents=(15,16,32)),SphereC28(Extents=(15,16,32)),SphereC29(Extents=(15,16,32)),CylinderEB0.0.0(Extents=(13,31,19)),CylinderEB1.0.0(Extents=(17,25,18)),CylinderCB0.0.0(Extents=(17,23,17)),CylinderCB1.0.0(Extents=(14,21,15)),CylinderEA0.0.0(Extents=(13,31,19)),CylinderEA1.0.0(Extents=(14,25,18)),CylinderCA0.0.0(Extents=(17,23,18)),CylinderCA1.0.0(Extents=(14,21,15)),FilledCylinderEB0(Extents=(12,11,25)),FilledCylinderEB1(Extents=(12,10,25)),FilledCylinderCB0(Extents=(12,9,21)),FilledCylinderCB1(Extents=(12,8,19)),FilledCylinderMB0(Extents=(14,11,25)),FilledCylinderMB1(Extents=(16,10,21)),CylinderSMB0.0(Extents=(14,27,15)),CylinderSMB1.0(Extents=(18,25,15)),FilledCylinderEA0(Extents=(12,11,25)),FilledCylinderEA1(Extents=(12,10,25)),FilledCylinderCA0(Extents=(12,9,21)),FilledCylinderCA1(Extents=(12,8,19)),FilledCylinderMA0(Extents=(14,11,25)),FilledCylinderMA1(Extents=(14,10,21)),CylinderSMA0.0(Extents=(14,27,15)),CylinderSMA1.0(Extents=(15,25,15))"""

# Split the string into individual items
items = data.split('),')

# # Function to parse each item
# def parse_item(item):
#     name, values = re.match(r'(\w+)\(Extents=\((.*?)\)', item).groups()
#     r, l, m = map(int, values.split(','))
#     return {'Name': name, 'R': r, 'L': l, 'M': m}

# # Parse all items
# parsed_data = [parse_item(item) for item in items]

# # Create DataFrame
# df = pd.DataFrame(parsed_data)

# # Set 'Name' as index
# df.set_index('Name', inplace=True)

# # Create the specific variables for SphereA2
# SphereA2_R = df.loc['SphereA2', 'R']
# SphereA2_L = df.loc['SphereA2', 'L']
# SphereA2_M = df.loc['SphereA2', 'M']

# print(df)
# print(f"\nSphereA2_R = {SphereA2_R}")
# print(f"SphereA2_L = {SphereA2_L}")
# print(f"SphereA2_M = {SphereA2_M}")

In [ ]:
items = data.split('),')
name,_,vals = items[0].split("(")
r,l,m=vals[:-1].split(',')
{
  name+"_R":r,
  name+"_L":l,
  name+"_M":m
}

In [ ]:
items[0].split("(")

In [ ]:
vals[:-1].split(',')

In [ ]:
folder_path = Path("/groups/sxs/hchaudha/spec_runs")
del_path = Path("/groups/sxs/hchaudha/spec_runs/del.sh")

with del_path.open('w') as f:
  for i in folder_path.iterdir():
    if i.is_dir():
      if "ID" in str(i):
        continue
      if "del" in str(i):
        continue
      f.writelines(str(i)+"\n")

In [ ]:
def write_dir(folder_path:Path,del_path_opened):
  for i in folder_path.iterdir():
    if i.is_dir():
      print(i)
      if "ID" in str(i):
        continue
      if "del" in str(i):
        continue
      if "Lev" in str(i) and i.is_symlink():
        continue
      if "Run" == i.name:
        del_path_opened.writelines(str(i)+"\n")
        return
      write_dir(i,del_path_opened)

folder_path = Path("/groups/sxs/hchaudha/spec_runs")
# folder_path = Path("/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eteq_tol_8")
del_path = Path("/groups/sxs/hchaudha/spec_runs/del.sh")
with del_path.open('w') as f:
  write_dir(folder_path, f)

In [ ]:
folder_path.name == "asd"